# 🔍 Introduction to Azure AI Search

 In this notebook, we explore **AI search** — a critical capability that combines information retrieval with artificial intelligence to enable smarter, context-aware search experiences.

Traditional keyword-based search systems often struggle with:
- Understanding the *meaning* behind user queries.
- Returning *relevant* results when vocabulary differs.
- Scaling across large, unstructured datasets (e.g., documents, emails, knowledge bases).

**AI Search** addresses these limitations by integrating:
- **Semantic understanding**: using embeddings to capture the meaning of text.
- **Hybrid retrieval**: blending traditional keyword matching with vector similarity.
- **Large Language Models (LLMs)**: enabling generative answers based on search context.

---

### 💡 What You'll Learn

In this notebook, you will:

- Understand different types of search: **keyword**, **vector**, and **hybrid**.
- Use **Azure AI Search** with **Azure OpenAI embeddings** to implement vector and hybrid retrieval.
- Apply a **Retrieval-Augmented Generation (RAG)** pipeline using **LLMs** to answer natural language questions based on enterprise content.

---

### 📦 Technologies Covered

- [Azure AI Search](https://learn.microsoft.com/en-us/azure/search/) — an enterprise-grade search engine with semantic and vector search.
- [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/) — for embedding generation and LLM-based responses.



In [19]:
import os
from dotenv import load_dotenv

load_dotenv()
import json
import wget
import pandas as pd
import zipfile
from openai import AzureOpenAI
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient, SearchIndexingBufferedSender
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryType,
    VectorizedQuery,
)
from azure.search.documents.indexes.models import (
    HnswAlgorithmConfiguration,
    HnswParameters,
    SearchField,
    SearchableField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
    ComplexField,
    CorsOptions,
    ScoringProfile,
    SearchResourceEncryptionKey,
)
from typing import List

from dotenv import load_dotenv
import os
from azure.identity import AzureCliCredential
from azure.search.documents import SearchClient
load_dotenv(override=True)
# Azure Search configuration
search_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME_HOTEL")
credential = AzureCliCredential()
# Create Search client instance
search_client = SearchClient(
    endpoint=SEARCH_ENDPOINT, index_name=INDEX_NAME, credential=credential
)

## Create an encryted index with Azure AI Search Python SDK

In this step, we use the **Azure AI Search Python SDK** to define and create a search index enriched with:

- **Semantic fields**: enable better understanding of document content and intent.
- **Vector search fields**: power similarity-based search using OpenAI-generated embeddings.
- **Customer-managed encryption keys (CMK)**: ensure your data is protected using your own Azure Key Vault key.
- **Complex and structured fields**: support faceting, filtering, and rich document modeling.

This index forms the foundation of our hybrid Retrieval-Augmented Generation (RAG) pipeline by supporting both keyword and vector-based retrieval.


In [24]:
credential = AzureCliCredential() # Authenticates by requesting a token from the Azure CLI. This requires being logged in via "az login", and will use the CLI's currently logged in identity.
# Get the service endpoint
endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT')
index_client = SearchIndexClient(endpoint, credential=credential)


# Define the fields for the index
fields = [
    SimpleField(name="HotelId", type=SearchFieldDataType.String, key=True, filterable=True),
    SearchableField(name="HotelName", type=SearchFieldDataType.String, sortable=True),
    SearchableField(name="Description", type=SearchFieldDataType.String),
    SearchField(
        name="DescriptionVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_profile_name="my-vector-profile"
    ),
    SearchableField(name="Category", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="Tags", type=SearchFieldDataType.Collection(SearchFieldDataType.String), searchable=True, filterable=True, facetable=True),
    SimpleField(name="ParkingIncluded", type=SearchFieldDataType.Boolean, filterable=True, sortable=True, facetable=True),
    SimpleField(name="LastRenovationDate", type=SearchFieldDataType.DateTimeOffset, filterable=True, sortable=True, facetable=True),
    SimpleField(name="Rating", type=SearchFieldDataType.Double, filterable=True, sortable=True, facetable=True),
    ComplexField(name="Address", fields=[
        SearchableField(name="StreetAddress", type=SearchFieldDataType.String),
        SearchableField(name="City", type=SearchFieldDataType.String, filterable=True, sortable=True, facetable=True),
        SearchableField(name="StateProvince", type=SearchFieldDataType.String, filterable=True, sortable=True, facetable=True),
        SearchableField(name="PostalCode", type=SearchFieldDataType.String, filterable=True, sortable=True, facetable=True),
        SearchableField(name="Country", type=SearchFieldDataType.String, filterable=True, sortable=True, facetable=True),
    ]),
    SimpleField(name="Location", type=SearchFieldDataType.GeographyPoint, filterable=True, sortable=True),
]


# Configure the vector search configuration
vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(name="my-hnsw-vector-config-1", kind="hnsw"),
            ExhaustiveKnnAlgorithmConfiguration(name="my-eknn-vector-config", kind="exhaustiveKnn")
        ],
        profiles=[
            VectorSearchProfile(name="my-vector-profile", algorithm_configuration_name="my-hnsw-vector-config-1")
        ]
    )

# Configure the semantic search configuration
semantic_search = SemanticSearch(
    configurations=[
        SemanticConfiguration(
            name="my-semantic-config",
        prioritized_fields=SemanticPrioritizedFields(
           title_field=SemanticField(field_name="HotelName"), 
           content_fields=[SemanticField(field_name="Description")], 
           keywords_fields=[SemanticField(field_name="Category")]
            ),
        )
    ]
)

customer_managed_key = SearchResourceEncryptionKey(
    key_name=os.getenv("KEYVAULT_KEY_NAME"),
    key_version="",
    vault_uri=os.getenv("KEYVAULT_URI"),
)

# Create the search index with the vector search and semantic search configurations
index = SearchIndex(
    name=os.getenv("AZURE_SEARCH_INDEX_NAME_HOTEL"),
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search,
    encryption_key=customer_managed_key,
)
# Create or update the index
result = index_client.create_or_update_index(index)
print(f"{result.name} created")

index_hotels created


## Create documents payload

In [29]:
# Create a documents payload
documents = [
    {
        "@search.action": "mergeOrUpload",
        "HotelId": "1",
        "HotelName": "Stay-Kay City Hotel",
        "Description": "This classic hotel is fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.",
        "DescriptionVector": [-0.048865054,-0.020307425,0.017633565,0.023875887,-0.04401433,-0.021689085,-0.04437217,0.011500583,0.03840817,0.00029058976,0.016907945,-0.009214383,-0.04512761,0.019889945,0.020973407,0.023040926,-0.026539808,0.050495215,0.07152826,-0.008786962,-0.009994673,-0.0053129313,-0.014601864,-0.048069853,0.021231845,0.022066806,-0.018021226,-0.010526463,0.07220418,0.0068685417,0.009472823,-0.023239726,0.040276892,0.03399481,0.0156058045,-0.001837658,-0.009567252,-0.03630089,0.009010613,0.027672967,-0.023398766,0.030078448,0.018428765,-0.006709502,-0.03598281,-0.018021226,-0.017782666,0.06655826,-0.019909825,0.010963823,-0.028428407,0.007325782,-0.030833889,-0.045724012,-0.0780489,0.024253607,0.018220024,-0.022762606,0.056777295,0.007817812,0.03355745,0.029163968,0.031967048,0.029959168,-0.051568735,0.057294175,-0.0156157445,0.03759309,-0.046002332,-0.020396886,0.053278416,0.016371185,0.03170861,-0.015685324,0.0010555041,0.024094567,0.0051886817,0.012872304,0.004055521,-0.03315985,-0.013568103,-0.023359006,-0.072243944,0.026480168,0.025068687,0.009010613,-0.018090805,-0.025207847,0.009408212,0.0025123358,0.024591567,-0.003725016,-0.0053924513,-0.025227727,-0.055385694,0.012136743,-0.011709323,-0.041310653,-0.021828245,0.04373601,0.030217608,0.023199966,-0.012912064,0.020277606,0.021609565,-0.031887528,0.014164504,-0.062264178,0.03315985,0.0034218458,-0.07550426,0.007653802,-0.04544569,-0.030973049,-0.0029298158,0.041708253,0.053198896,-0.03379601,-0.010834603,0.025168087,-0.031569447,-0.023836127,-0.025088567,-0.009935033,0.0017009829,-0.03395505,0.03174837,-0.030814009,-0.0155958645,-0.0030192758,0.009477792,-0.024830127,-0.046757773,0.0055216714,-0.015069044,0.024015047,0.015735025,-0.020655327,-0.020357126,0.015287724,0.003705136,-0.03389541,-0.026142208,-0.041390173,-0.03705633,0.06818842,0.03186765,0.007181652,-0.012802724,0.030694729,0.025366887,0.064729296,0.029680848,-0.011639743,-0.0016351305,0.0029944258,0.021788485,-0.017921826,-0.03486953,0.040992573,-0.021629445,0.03576413,-0.07232346,0.004868116,0.055783294,0.031112209,-0.046121612,-0.049262654,-0.04500833,-0.023021046,0.03538641,-0.020536046,0.006500762,0.031808008,0.03359721,0.052920576,-0.017812485,-0.014949764,0.028845888,0.019780606,0.019999286,-0.020874007,0.0865973,-0.057691775,0.019442646,0.03190741,-0.079122424,0.046519212,0.018170325,0.012196383,0.013448824,0.009865453,-0.0850069,0.0057204715,-0.03270261,0.051727775,-0.03242429,-0.041151613,0.012902124,0.0003308157,-0.011937943,0.0045102765,0.018617624,-0.016401004,-0.018369125,0.009716352,0.0052185017,-0.024850007,0.019880006,0.03294117,-0.004353721,-0.04373601,0.019134505,0.0693017,-0.016222084,-0.03570449,-0.050018094,0.003702651,-0.028448287,0.047791533,0.00023576444,0.0012723204,0.0047712014,0.028030807,-0.026162088,0.06846674,-0.0069281817,-0.025963288,-0.004067946,0.011848483,0.0010604741,-0.013090984,-0.024174087,-0.029541688,-0.014224144,0.04238417,0.007236322,0.0034392409,-0.03447193,-0.013001524,-0.03357733,0.007017642,-0.008697502,0.011450883,0.030058568,-0.019154385,-0.014104864,-0.022822246,-0.011013523,0.024631327,-0.0059391516,0.03238453,0.03644005,-0.028925408,0.020774607,-0.0029447258,0.0016053105,0.015426884,0.041946813,0.025426527,0.019094745,-0.000408472,0.056061614,-0.024492167,-0.012385244,-0.046996333,-0.054868814,0.030694729,0.00025517851,-0.059918337,-0.045843292,0.0029571508,-0.0068486617,-0.03745393,0.03638041,-0.031092329,0.0055167014,0.000035877198,-0.042145614,-0.0138861835,-0.022086686,-0.03785153,0.07232346,-0.013031344,-0.018657384,-0.006461002,-0.013826543,0.029422408,-0.023716846,0.007141892,-0.0025309732,0.0026788306,0.011659623,-0.03838829,-0.00011531956,-0.007922182,0.022881886,-0.06938122,0.002265078,-0.0021681632,-0.023736726,0.0750669,0.03610209,-0.014820544,-0.018041106,0.061429217,0.003287656,-0.029800128,0.020436646,0.022941526,-0.0022812306,0.020237846,-0.019184206,-0.0716873,-0.022066806,-0.039879292,0.014701264,-0.0058447216,-0.032245368,0.0060137017,0.010049343,-0.021470405,-0.0050147315,0.007718412,0.057413455,-0.023657206,0.011798783,0.025943408,-0.009199472,-0.0021818306,0.040952813,-0.032682728,0.018190205,-0.0026639206,0.022444526,0.016629625,-0.015466644,-0.014800664,0.024512047,0.0016475555,0.014512404,-0.058327936,-0.012653624,-0.010049343,0.064331695,-0.025983168,-0.010337603,-0.017971525,-0.013677443,-0.010993643,-0.056817055,-0.027593447,-0.009542403,0.010009583,0.014422944,0.014850364,0.007609072,0.054550733,-0.011073163,0.039839532,-0.024452407,-0.024929527,0.017822426,-0.007151832,0.014760904,0.007256202,-0.045724012,0.009646772,-0.027692847,0.017395005,-0.007678652,0.0056459215,0.013220204,0.009607012,-0.064013615,0.017116684,-0.001591643,0.008886362,-0.04234441,-0.041310653,-0.0020849155,-0.04294081,0.013478644,-0.028388647,-0.010526463,0.022265606,-0.004798536,-0.014870244,-0.027573567,0.057015855,0.04492881,-0.011560223,0.0049749715,-0.008364513,-0.011540343,0.010228263,0.015029284,0.052960336,-0.021331245,-0.0029397558,0.058407456,-0.04341793,-0.011083103,-0.022524046,0.03178813,-0.014661504,-0.03381589,-0.055226654,-0.0069679418,0.0030714609,-0.03801057,-0.023796367,0.008453973,-0.019015225,0.024273487,0.007400332,0.04262273,-0.026818128,-0.0122659635,0.004773686,0.047155373,0.03572437,0.03767261,0.03482977,0.012941884,0.018597744,0.0012580316,-0.012216263,-0.07781034,-0.03226525,0.015019344,-0.0059987917,0.023120446,0.057135135,-0.019989345,0.060196657,-0.020416766,0.012434944,-0.024392767,-0.021072807,-0.057612255,-0.016659444,-0.04242393,-0.03741417,0.023040926,0.051250655,0.012434944,0.062184658,0.011480703,0.0044829412,0.0021992256,-0.017723026,-0.020327305,0.021132445,-0.03870637,0.07005714,0.019383006,-0.016540164,0.03753345,-0.03630089,0.018786605,-0.010755083,-0.022186086,-0.0048109614,0.052284416,0.023418646,-0.011997583,0.017563986,0.024671087,-0.013130744,-0.04520713,-0.030376649,0.04298057,0.04266249,0.018667325,-0.03809009,-0.029621208,0.053397696,0.041787773,-0.018776665,0.015347364,-0.004224501,-0.03628101,-0.030654969,-0.024333127,-0.004423301,-0.050813295,0.021689085,-0.002314778,0.016748905,0.008260142,0.020476406,0.052801296,-0.004714046,0.015814545,-0.0106358025,-0.049103614,-0.027513927,-0.030913409,-0.016301604,-0.008329722,0.040833533,0.03230501,-0.050614495,0.014492524,0.03178813,-0.011361423,-0.024472287,0.017375125,-0.018895945,-0.032245368,0.017544106,-0.017315485,0.010437003,-0.024333127,0.020317366,0.010685503,0.012484644,0.011629803,-0.015874185,0.009602043,0.001994213,-0.023895767,0.030575449,0.036121972,0.039680492,-0.03906421,-0.028726608,0.026818128,0.027275367,-0.018548045,-0.03502857,-0.014949764,0.03822925,0.010347543,0.0016599805,-0.009626892,-0.021708965,0.011669563,0.0053328113,0.060554497,0.003690226,-0.0015096379,-0.030257368,-0.026261488,-0.0060137017,-0.03385565,-0.010054313,0.024909647,-0.024154207,0.011232203,0.0012617591,0.008881393,0.03846781,0.014939824,0.009318752,0.048030093,-0.016351305,-0.014234084,0.019621566,-0.055504974,0.023677086,-0.021013167,0.001827718,0.03508821,0.003188256,0.025585568,0.04262273,0.0028428407,-0.026639208,0.023319246,-0.014353364,-0.014661504,-0.003752351,0.0155958645,0.010526463,0.012096983,0.010745143,0.013498524,0.049739774,0.03357733,-0.016162444,0.023279486,0.021152325,-0.04298057,0.013975644,0.018329365,0.025187967,-0.017563986,0.03592317,0.006396392,-0.007166742,0.04512761,0.0057055615,-0.010675563,0.006267172,-0.04302033,-0.0021656782,-0.03371649,0.026838008,-0.028567567,0.0015270329,0.003747381,-0.011102983,0.004674286,0.025963288,-0.008419182,0.023100566,0.010397243,-0.025923528,0.012723204,0.0028030807,0.022782486,0.011699383,-0.029979048,0.03618161,-0.0041524363,-0.007822782,0.019363126,-0.026678968,-0.048825294,0.041787773,0.03250381,-0.0009753628,0.0054620313,0.017086865,-0.011679503,-0.022245726,-0.010153713,-0.029641088,0.013289784,0.023696966,0.048109613,-0.003876601,-0.0017991405,0.020714967,0.03528701,-0.027494047,-0.017822426,-0.000420897,-0.018418824,0.024531927,0.040634733,0.028150087,-0.010755083,-0.008439062,0.04461073,0.007499732,-0.025863888,0.030555569,-0.028746488,0.0027633207,-0.0028080507,-0.008106072,-0.0053924513,0.0058099316,0.031450167,0.023498166,0.011490643,-0.003946181,0.007385422,0.026639208,-0.04234441,0.0034119058,-0.047274653,0.0052980212,0.0026142208,0.025207847,-0.010755083,0.012762964,0.031132089,0.012474704,0.022265606,0.04282153,0.007355602,0.0097561125,-0.016609745,-0.058009855,-0.020893887,-0.029959168,-0.000963559,0.017255845,-0.020635447,-0.009129892,0.023597566,-0.023677086,-0.011659623,0.026420528,0.024929527,0.0015096379,0.007286022,0.016013345,0.029342888,-0.012494584,-0.015387124,-0.046638492,0.0005808689,-0.009020553,-0.010506583,-0.0022961407,-0.0017320454,-0.019720966,0.015188324,-0.020734847,0.001775533,-0.0022439556,0.029919408,-0.003777201,0.06484858,-0.003720046,-0.008150802,-0.015516344,0.013786783,-0.06635946,0.007117042,-0.0049724863,0.00073990895,0.011102983,-0.020098686,-0.0044879112,-0.017116684,0.028547687,0.0058149016,-0.012603924,-0.03693705,-0.020317366,-0.0024029957,-0.020933647,-0.024392767,-0.003772231,0.030654969,0.010456883,-0.030873649,-0.03606233,-0.008230322,-0.000918829,-0.023696966,0.001904753,-0.04512761,-0.013438884,-0.025585568,-0.029183848,-0.03375625,0.010327663,0.03242429,-0.009050372,0.008160742,-0.03427313,-0.049183134,0.029422408,-0.021987285,0.0025819158,-0.011262023,0.014959704,-0.0020724905,-0.030953169,0.03453157,-0.0027384707,-0.011331603,0.060156897,0.008339662,0.00091448025,-0.039322652,-0.0013742053,0.03226525,-0.03321949,-0.002004153,0.06556426,-0.008946002,-0.023875887,0.0012238629,0.0013829028,0.013717203,-0.022961406,0.026241608,-0.024452407,-0.028189847,-0.020039046,-0.0018836305,-0.021430645,0.0052532917,-0.018488405,0.011301783,-0.0059292116,0.018319424,0.008066312,-0.0002029935,-0.024730727,0.010745143,0.023836127,0.022365006,-0.018518224,0.022702966,-0.007350632,-0.0015531254,-0.014025344,0.015407004,-0.017275725,-0.0014524829,0.006441122,0.003921331,0.0054570613,-0.010715323,-0.032483928,-0.0026564656,0.03584365,-0.0061429217,0.012951824,0.030992929,-0.022504166,0.00093436026,0.017335365,0.04508785,0.03492917,0.007131952,-0.0028826008,-0.014442824,0.018269725,0.021271605,-0.016053105,-0.020108625,-0.029760368,-0.0052334117,0.040833533,0.002319748,0.0068337517,-0.014780784,0.006769142,0.0054471213,-0.00095424027,-0.03910397,0.023975287,-0.015198264,0.016381124,0.022702966,-0.025923528,0.002024033,-0.009621923,-0.004868116,-0.019899886,0.019522166,0.028329007,-0.052284416,-0.0010331391,-0.027474167,0.014830484,0.008906242,0.03604245,-0.023557806,0.023359006,0.0021731332,-0.008488762,0.03659909,-0.019750785,-0.016361244,0.011729203,-0.026559688,-0.015814545,0.010516523,-0.018180264,-0.065047376,0.011550283,0.007619012,-0.07443074,-0.006709502,0.03789129,-0.007305902,-0.027692847,-0.008583193,0.014949764,-0.032006808,0.020078806,0.061667778,0.014015404,0.021788485,0.0029571508,-0.0005147058,0.0044307564,-0.015645564,-0.07121018,0.007355602,-0.014413004,0.03286165,0.015804604,-0.0015481554,0.030674849,-0.03640029,0.0097561125,-0.017673325,-0.013796723,0.03387553,0.019790545,0.026022928,0.009010613,-0.025744608,-0.0002453938,-0.022842126,0.058447216,-0.03719549,0.027633207,-0.006774112,-0.012146683,0.016768785,-0.0045127613,0.009462883,0.039362412,-0.03238453,-0.004000851,0.020416766,0.018130565,0.0082551725,0.003792111,-0.008220382,-0.0090354625,0.016033225,-0.00014125675,-0.0026092508,0.015715145,-0.0044009364,-0.018776665,-0.011202383,0.019522166,-0.052841056,-0.00079892774,0.041668493,0.03737441,-0.013230144,0.021132445,0.004219531,-0.04278177,-0.018886006,-0.021251725,0.024154207,-0.009656712,0.04274201,0.014611804,0.001763108,0.0098058125,-0.0060236417,-0.0017519254,0.010864423,0.020675207,-0.021033047,0.003993396,0.014452764,-0.03993893,0.025466288,0.022524046,-0.008598102,0.000807004,-0.0023185057,0.017295605,-0.012574104,-0.023796367,0.012822604,0.025943408,-0.004463061,0.0006352283,0.008439062,-0.008886362,-0.03353757,0.022563806,-0.000978469,-0.0009703928,0.0013207778,-0.00025067443,-0.047513213,0.06437146,0.010407183,0.0008132165,-0.0051290416,0.029959168,-0.0090652825,-0.001760623,0.019035105,0.04329865,0.021887885,0.025227727,0.0029273308,0.030654969,-0.024810247,-0.022325246,0.024551807,-0.028229607,0.031291127,0.029303128,0.03347793,0.020595687,0.022643326,0.030615209,0.03379601,0.006272142,0.016152505,0.0015730055,-0.03451169,0.019293545,0.009159712,-0.0017071954,-0.011431003,0.03816961,-0.027454287,0.024850007,-0.016798604,0.019333305,-0.0047314414,0.03447193,-0.006421242,0.023557806,0.029004928,-0.022126446,0.029124208,0.016639564,0.023060806,0.009527492,-0.047234893,0.008667682,0.029521808,0.014104864,-0.008444033,0.019273665,0.022484286,-0.012504524,0.030118208,0.0024005107,0.0024676058,-0.0082054725,0.026142208,0.010198443,-0.001658738,0.021172205,0.023995167,-0.009701443,0.0025471258,0.03282189,0.0058049615,0.0027956257,0.028428407,-0.0034243308,-0.0047960514,0.024193967,-0.0139160035,0.012802724,-0.008369482,-0.011311723,-0.031768247,0.032762248,0.0012226204,0.019780606,0.010496643,0.006510702,-0.009244203,-0.0060385517,-0.007748232,-0.015844364,-0.020834247,0.0068586017,0.012057223,-0.028309127,0.009000673,-0.0058745416,0.015665444,0.007161772,0.024969287,-0.040754013,-0.024034927,0.012464764,-0.041072093,0.0082154125,-0.014711204,0.040754013,-0.006441122,0.015377184,-0.03797081,-0.024571687,0.03170861,-0.017285665,-0.026122328,-0.0013307178,-0.010794843,0.006674712,-0.004239411,0.052204896,-0.023975287,-0.0023011107,-0.03347793,0.03162909,0.03651957,-0.024512047,0.016560044,0.007763142,0.003367176,0.018647445,-0.03323937,0.0077929622,0.015695265,0.004028186,0.011083103,0.016092865,-0.014263904,0.030953169,0.002411693,-0.013627743,-0.017096804,-0.0105960425,-0.031151969,0.008752173,0.007499732,-0.03703645,-0.04524689,-0.017454645,-0.009318752,0.016331425,0.017941706,0.052761536,0.011331603,0.058168896,0.049103614,0.022424646,0.03250381,0.009537432,-0.010934003,-0.021569805,0.007236322,0.018895945,-0.0015779755,-0.020993287,-0.008970853,-0.029462168,0.008150802,0.012842484,0.013200324,0.00045506575,-0.011778903,0.011540343,0.003225531,0.027752487,-0.009522523,0.013727143,0.014681384,0.010516523,-0.007867512,0.017216085,-0.0047910814,0.003976001,0.03735453,-0.025068687,0.04361673,0.04321913,0.029541688,-0.0015208204,0.0051290416,-0.0010256841,0.026500048,-0.00057931576,0.023160206,0.0090354625,-0.017454645,-0.04345769,-0.001827718,-0.023498166,-0.008125952,0.006217472,0.025486168,-0.019432705,0.010407183,0.032006808,-0.03250381,0.019373065,0.007107102,-0.006585252,0.049183134,0.023597566,0.026460288,0.021271605,0.019263726,0.012087043,0.010864423,0.029601328,0.006490822,0.006341722,-0.00012300753,-0.0009902727,0.020476406,-0.03453157,0.026539808,-0.023796367,-0.0057950215,-0.003570946,-0.013776843,0.00049513637,0.001666193,0.047195133,0.020377006,-0.023875887,0.003623131,0.025724728,-0.029482048,-0.007156802,-0.012832544,-0.016609745,-0.011212323,0.049501214,-0.010034433,-0.010456883,-0.013329544,0.017335365,-0.023438526,0.008657742,-0.007723382,-0.006490822,0.0023359007,0.0059640016,-0.0138762435,-0.0053775413,0.03479001,-0.029243488,-0.011609923,-0.006242322,0.00025347006,-0.04294081,-0.025187967,0.029482048,0.03759309,0.013587983,0.018687205,-0.03371649,0.017275725,0.000058242204,0.007102132,0.007276082,-0.009790903,-0.026122328,-0.03310021,0.014174444,0.019482406,0.030018808,0.013458764,0.03622137,-0.029641088,0.017166385,-0.011371363,0.03914373,0.026360888,0.018995345,-0.006366572,-0.027036808,-0.015913945,0.023776487,0.006719442,-0.018150445,0.007271112,-0.012882244,-0.017295605,-0.026102448,0.015735025,-0.003754836,0.0048258714,-0.022404766,-0.040913053,0.016251905,-0.030217608,-0.028309127,-0.04397457,-0.03504845,-0.012395184,-0.039402172,-0.019542046,0.03584365,-0.020555926,0.0053029913,-0.030217608,0.013170504,-0.017424826,0.04357697,0.008439062,0.012385244,0.021410765,0.040594973,0.009989703,0.021748725,0.041429933,-0.019204086,-0.013737083,-0.0055564614,0.025068687,0.04286129,0.00016866942,0.025048807,-0.014234084,0.0015904005,-0.024472287,-0.003556036,-0.0011983915,-0.029541688,-0.030237488,0.012444884,-0.039760012,0.000839309,0.022365006,0.028786248,-0.03315985,0.025744608,-0.007067342,-0.012643684,-0.026102448,0.001646313,0.019243846,-0.023875887,-0.010735203,-0.017474525,0.0077929622,0.04425289,-0.026181968,-0.017305546,-0.018687205,0.032086328,0.009070252,0.017872125,0.003565976,0.0025844008,0.011599983,0.027692847,0.023140326,-0.004130071,0.03496893,0.018051045,-0.03437253,-0.011122863,0.0053825113,-0.012534344,0.007837692,0.030515809,0.006207532,-0.03292129,-0.011252083,0.009830663,-0.010228263,0.013468704,-0.011878303,-0.008125952,0.04317937,-0.04536617,0.003700166,-0.016440764,0.020635447,0.0038964811,-0.028070567,-0.015049164,-0.0051340116,-0.0041027362,-0.004299051,-0.054630253,-0.0048879962,0.0139259435,0.017116684,0.010387303,0.012872304,-0.019780606,-0.0017892005,0.009020553,0.010218323,0.032642968,0.004818416,-0.026380768,0.0058248416,0.040952813,-0.021828245,-0.014452764,0.0011344028,-0.03194717,0.007857572,-0.012067163,-0.0047264714,0.0022153782,-0.0030764309,-0.021052927,-0.015983524,0.00041685888,-0.022464406,0.0089957025,-0.03974013,0.0022663206,0.052284416,-0.020436646,0.026758488,-0.0027484107,0.0056757415,-0.0058795116,0.011093043,0.023199966,-0.00015049786,0.012444884,0.03296105,-0.010536403,-0.011321663,-0.020029105,0.007991762,-0.041867293,0.025863888,-0.003715076,0.016490465,0.014333484,0.0034566359,0.029998928,-0.003106251,-0.006257232,-0.027613327,-0.023239726,-0.013230144,-0.054113377,0.03447193,-0.008001702,0.012037343,0.009597072,0.017554045,-0.0021905282,-0.001618978,0.0082651125,-0.0011232203,-0.007927152,-0.018478464,-0.009974793,-0.019303486,-0.039600972,-0.0155859245,0.030476049,0.017226025,-0.024313247,0.010039403,0.004234441,-0.018707085,0.011023463,-0.018488405,0.016798604,0.04373601,-0.024710847,0.0069281817,0.0028279307,0.004015761,-0.009085163,0.0030590359,-0.0059242416,-0.010705383,-0.012653624,0.000961074,0.011142743,-0.003352266,0.047871053,-0.04544569,-0.0015233054,0.023875887,0.00043021573,0.007474882,-0.017206145,-0.004443181,-0.014035284,0.018001346,0.0011685715,-0.020635447,0.0014760904,0.027832007,0.015397064,-0.031489927,0.015327484,0.014015404,0.013518404,0.028905528,-0.024690967,0.024810247,-0.026181968,-0.015317544,0.010993643,0.011450883,0.030297128,0.009920123,-0.020068865,0.0053029913,0.015108804,0.0007604102,-0.019144446,-0.017365186,0.003215591,0.0025148208,-0.023975287,0.004709076,-0.021390885,-0.012027403,0.024869887,-0.028905528,-0.03638041,0.0043015364,-0.029859768,0.006232382,-0.0069629718,-0.0024464831,0.013120804,0.031231489,0.0082154125,0.0013232628,0.0028552657,-0.009219352,0.019144446,-0.006774112,0.0034094208,-0.03500869,0.026221728,-0.026718728,0.03735453,0.0155958645,-0.0059242416,-0.023677086,-0.014691324,0.013359364,-0.004299051,0.016500404,0.009209412,0.004085341,-0.010774963,-0.004738896,0.016490465,-0.027315127,0.021033047,-0.03445205,0.031092329,0.006749262,-0.008806842,-0.008056372,-0.04441193,0.014184384,-0.0155859245,0.016560044,0.007375482,0.011440943,-0.026162088,-0.018120624,-0.012772904],
        "Category": "Boutique",
        "Tags": [
            "view",
            "air conditioning",
            "concierge"
        ],
        "ParkingIncluded": "false",
        "LastRenovationDate": "2022-01-18T00:00:00Z",
        "Rating": 3.60,
        "Address": {
            "StreetAddress": "677 5th Ave",
            "City": "New York",
            "StateProvince": "NY",
            "PostalCode": "10022",
            "Country": "USA"
        },
        "Location": {
            "type": "Point",
            "coordinates": [
                -73.975403,
                40.760586
            ]
        }
    },
    {
        "@search.action": "mergeOrUpload",
        "HotelId": "2",
        "HotelName": "Old Century Hotel",
        "Description": "The hotel is situated in a nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts. The hotel also regularly hosts events like wine tastings, beer dinners, and live music.",
        "DescriptionVector": [-0.04683398,-0.01285595,0.03386663,-0.015239983,-0.0033393162,-0.014727527,-0.012042706,-0.011630513,0.0024954358,-0.037431534,0.006550519,0.021155503,-0.06024695,-0.036050133,0.0026764662,0.036094695,-0.06069256,0.014025685,0.052270465,0.01747919,-0.020620765,-0.017501472,-0.04121925,-0.07085255,0.01518428,0.013591212,-0.06412379,0.050488014,0.020865854,0.05596906,-0.001694724,-0.020999538,0.0724122,0.038099956,-0.023907166,-0.055077832,0.015050597,0.011842179,0.0164877,0.014359896,-0.032730315,0.012087267,0.01220981,-0.011463407,-0.00083482906,-0.027271548,-0.024285937,0.049953274,-0.0077592456,0.0072412197,-0.04284574,0.006394554,-0.0024355564,-0.0005716386,-0.039369956,0.035426274,-0.008778586,-0.04821538,0.057439584,0.011274022,0.055612568,0.0020456447,0.06715396,0.016209193,-0.06875817,0.031839088,-0.026491724,0.029811544,0.016342876,-0.009335604,0.038345043,0.036339782,-0.0041637016,-0.043313634,-0.03622838,-0.026825935,0.0059099495,-0.00022872507,-0.004712363,0.015540771,-0.066619225,-0.00857806,-0.034980662,0.07210027,0.014215072,-0.0058709583,-0.0051830425,0.011909021,-0.030903298,0.04516293,0.044739597,0.046611175,0.034133997,0.0056509366,0.015451649,-0.017356647,0.0119647235,-0.009001393,-0.013245862,0.06327712,0.047235034,-0.058108002,0.021589976,-0.012310074,-0.011418846,-0.00444221,0.015195421,-0.029922947,0.014816649,-0.02078787,-0.078160614,-0.030212596,-0.067822374,-0.009346744,-0.028274179,-0.011246171,0.028318739,-0.016298315,-0.018660067,-0.0020957761,-0.005776265,0.000048216774,-0.06185116,-0.009981743,-0.0084499465,-0.0759771,-0.014772088,-0.049596786,-0.008600341,-0.021122081,-0.019595854,-0.0056481515,0.035493117,-0.013535511,-0.027561197,0.03125979,0.0224255,0.021545414,-0.034557328,-0.014315334,-0.0031749962,0.023639798,-0.023483833,-0.02272629,0.0036484606,0.014025685,0.031059263,0.00037354947,-0.032752592,0.0514238,0.00067712367,0.040261183,0.01584156,0.0109453825,-0.034846976,0.012555161,0.01847068,-0.008967972,-0.037698902,0.04915117,-0.031616278,0.0072690705,-0.025912426,0.023684358,0.01418165,0.04901749,-0.03792171,0.0058709583,-0.03359926,0.022146992,0.006812317,0.016721647,-0.030524526,0.064480275,0.04313539,0.04870556,0.006333282,0.005478261,0.019740678,0.010817268,0.002482903,0.01021569,0.07824974,-0.008444376,-0.027382951,0.02108866,-0.09473743,0.010132138,-0.004305741,0.014738667,-0.012900512,-0.01418165,-0.020119451,0.031393472,-0.023171904,-0.009864769,-0.021601116,0.028942598,-0.002686214,-0.010778277,-0.06465852,-0.06670834,0.009920471,-0.0036985923,-0.007642272,-0.042778898,0.016231472,-0.0059712213,-0.014560422,0.0007401362,-0.0013333592,-0.030190317,0.03400031,0.026536286,0.0050215074,-0.03658487,-0.0626087,0.031081542,-0.0051635467,0.050220642,-0.020643046,-0.017490331,-0.020921554,0.02802909,-0.017211823,0.0021319822,-0.006823457,-0.025154883,-0.024196815,-0.008940121,-0.013346125,-0.03660715,-0.02108866,0.000490523,-0.012566301,0.004812626,-0.060870808,0.0074974475,0.03368838,0.049284857,-0.033777505,-0.011842179,0.02898716,-0.0062664403,0.0500424,0.0040912894,-0.019562434,-0.04580907,0.030546807,0.03680768,-0.053161692,0.015685596,-0.0113074435,-0.010043015,0.02996751,0.005514467,-0.03883522,-0.009636393,-0.012466039,-0.012544021,0.08306236,0.0016195267,-0.030101193,-0.004511837,-0.005776265,-0.034044873,-0.00019164862,0.040238902,-0.0070406934,-0.02704874,-0.027160143,-0.017935945,-0.039748725,0.0021528704,0.0055590286,-0.029276809,0.004347517,-0.019818662,0.017724277,-0.014627264,-0.019016556,-0.061762035,0.037097327,0.014393317,-0.040350303,0.043380477,0.04901749,0.0063555627,-0.009981743,-0.005447625,0.025600497,-0.024308218,0.006723194,-0.002704317,-0.0064781066,0.017624015,0.01584156,-0.02736067,-0.03154944,-0.009296612,-0.03197277,0.024330499,0.040706795,0.010410646,-0.03417856,0.036651712,0.0021166643,0.000062229235,0.02204673,0.021188922,0.032195576,0.012443758,0.009313323,-0.055879936,-0.028385581,0.002272629,0.0013486772,0.025823304,-0.013067616,-0.03600557,0.007480737,0.005865388,0.005015937,0.01747919,0.03357698,-0.0729915,-0.0030496675,0.0073024915,-0.015696736,-0.039459076,0.0034451496,-0.01915024,0.013914282,-0.027249267,-0.038679253,0.040149778,-0.0066674924,0.07464027,0.02666997,-0.025979267,-0.01053876,-0.055612568,-0.034111716,-0.019584714,0.05641467,-0.047279596,-0.007959772,-0.039035745,0.042110477,0.03567136,-0.06697571,-0.012466039,0.015306825,0.04382609,0.029120844,-0.0013598176,0.051824853,0.03785487,-0.08234938,0.0045563984,-0.035203468,-0.028452424,0.0016195267,0.0119424425,-0.01088411,-0.0051273406,-0.009981743,-0.006450256,-0.014738667,-0.04514065,-0.03894662,-0.027850846,0.0048683276,-0.00792078,-0.05904379,0.037721183,0.030101193,0.0001987854,0.0069961324,-0.042444687,0.032039613,-0.0029689001,0.012588582,-0.037075046,0.035805047,0.0019161381,0.023127342,-0.00824385,0.0041748416,0.027182424,0.019707259,-0.04710135,-0.01714498,-0.03890206,0.02370664,-0.021266906,0.034089435,-0.006227449,-0.026892776,-0.011831039,0.017501472,-0.039102588,0.0019314562,-0.048660997,-0.028073652,-0.04449451,0.014883491,-0.008656043,0.0073916144,0.028786633,-0.027895406,-0.011235031,0.01683305,-0.009848059,-0.013736037,-0.015596474,0.025801023,-0.04342504,0.00024056167,0.027405232,0.04683398,0.028764352,0.022336379,0.013568932,0.0010987158,-0.009318893,-0.031616278,-0.03569364,-0.055256076,-0.040060654,-0.00070358196,-0.020810151,-0.015373667,-0.012889371,-0.04378153,0.0659508,-0.050220642,0.00956955,0.010633453,0.0067733256,-0.050621696,-0.012800248,0.005043788,-0.029945228,0.007514158,0.054498535,0.013312704,0.07504132,-0.00010800906,-0.036540307,-0.0077592456,0.00039513386,-0.024642428,-0.021679098,0.0028004025,0.0026639334,0.017701996,0.012388056,0.00957512,-0.017122699,0.0035844038,-0.03584961,-0.023751201,0.017222963,0.02278199,-0.036094695,-0.03847873,0.074194655,0.01155253,-0.05659292,-0.029410493,-0.016632525,0.061762035,0.023149623,0.0061661773,-0.013502089,0.0030524526,0.05997958,0.0049184593,0.012889371,0.019495592,0.009508278,0.0020512147,-0.00428903,-0.012633143,-0.023951726,-0.042021357,-0.000070279864,-0.028251898,-0.015752438,-0.0052638096,-0.007998763,0.008004333,-0.0078817895,-0.001859044,0.037052765,-0.029031722,-0.02435278,-0.023550674,-0.0026639334,0.0081992885,0.012978494,0.020654187,0.016120069,0.02566734,0.06568343,-0.0031443604,-0.001451029,0.041085567,-0.004043943,-0.007798237,-0.020598484,0.003392233,0.0113965655,0.029254528,0.014850071,0.019127961,0.014359896,-0.021356028,-0.013758318,0.016231472,0.018660067,-0.01418165,0.011162619,0.0033198209,0.041018724,-0.029856106,-0.060514316,-0.008422095,-0.021578835,-0.011246171,-0.013669195,0.00069139723,-0.023216464,0.019261645,-0.013914282,0.018715767,0.014783229,0.033799786,0.004742999,0.038099956,0.018504102,-0.037498377,-0.018069629,-0.015607614,-0.019339627,-0.023327868,0.0025650628,-0.009307752,0.0030134614,0.05841993,0.014215072,-0.038345043,0.0022489557,-0.0092242,-0.0062441593,0.023149623,0.045051526,0.0026179794,0.045385737,-0.015574193,0.0048989635,-0.004999227,0.036362063,0.02602383,-0.0269819,0.007436176,0.05075538,0.01681077,-0.00008929677,-0.019306205,0.020353397,-0.0048544025,0.043514162,0.006021353,0.029455055,0.0010799165,0.0044867713,-0.013624634,0.018225593,0.022291817,-0.0060436334,0.016342876,0.020030327,-0.019963486,-0.015897263,0.06523782,0.049685907,0.01549621,0.02936593,0.010020734,-0.0016989015,-0.0031248648,-0.028318739,0.0230605,-0.005166332,-0.0051440513,0.007319202,-0.030791894,0.068045184,-0.014738667,-0.0230605,0.006879159,0.009803497,-0.0082215695,0.04585363,-0.0029772553,-0.017189542,-0.010705865,-0.025645059,-0.00084318436,-0.017111558,-0.008739595,-0.04875012,-0.04121925,0.00010339626,-0.035649084,-0.010583322,0.014983755,-0.008979113,-0.0077536753,0.010605602,0.008383105,0.02666997,0.0066117905,0.031036982,0.013301563,-0.002193254,-0.014560422,0.025221726,0.042868022,0.016242612,0.03092558,-0.024441902,-0.028920317,0.035181187,0.03892434,-0.0054337,0.010004024,-0.02671453,0.0079040695,0.019250505,0.030212596,0.00004560576,0.009731085,0.025132602,0.02929909,0.020921554,0.018927434,-0.01055547,-0.029276809,0.028942598,0.03560452,-0.006032493,0.0017685287,-0.0056592915,-0.032685753,-0.011040075,-0.007970911,0.025466813,0.00892341,0.054765902,-0.012120687,-0.01810305,0.0019091754,0.020119451,-0.0013347517,-0.00230048,-0.05240415,0.03304224,0.0718329,0.03195049,0.04776977,0.02435278,0.03137119,0.013680335,-0.017378928,-0.026803654,-0.00013011567,-0.010393935,-0.003963175,0.018203313,-0.02961102,0.0041637016,0.014916913,0.0039325394,-0.020954976,0.06880273,0.022291817,0.00724679,-0.00088426436,0.0041024294,0.008906701,0.000018549097,0.035827328,-0.03166084,0.00888442,-0.02466471,-0.02802909,-0.04222188,0.018983137,0.0062720105,-0.0045731086,-0.014549281,0.012020425,0.011630513,0.021835063,-0.014749807,0.030769615,0.013791738,0.025845584,-0.02867523,0.00033890997,-0.0050215074,0.049195733,0.03569364,-0.024575585,0.013903142,-0.0052220332,-0.017189542,0.021055238,0.00956955,-0.013969984,-0.043581,-0.021333747,-0.014939194,-0.0018270154,-0.009719945,-0.015262263,0.05828625,-0.009853629,0.0049936567,0.0058375373,-0.0027015319,0.053161692,-0.01483893,0.03362154,0.011190469,-0.008082315,0.005639796,-0.014226212,-0.024686988,-0.0047959154,-0.03322049,0.024597866,-0.044717316,0.016086647,-0.020542784,-0.05690485,0.025600497,-0.058152564,0.007051834,0.019930065,0.0049546654,-0.0024230236,-0.03988241,0.022536904,0.062163085,0.006160607,0.021211203,-0.012933932,0.008327403,-0.006121616,0.029232247,0.003924184,-0.007447316,0.04516293,0.013658054,-0.022692868,-0.04349188,0.016053228,-0.022169273,-0.010483058,0.010360515,-0.0006656352,-0.013346125,0.027494354,-0.038411885,0.0060436334,-0.015975244,0.031415753,-0.030747334,-0.017746558,0.0052276035,0.026313478,-0.007157667,-0.030479966,-0.015507351,0.016710507,-0.0035426274,0.0055534584,-0.023974007,0.010772707,-0.0064224047,-0.0016042087,0.015540771,0.03400031,-0.01615349,-0.027115583,0.023327868,0.015407087,-0.014515861,-0.0151174385,-0.02209129,-0.0001199327,0.026424881,0.01151911,0.024129972,0.00826613,-0.022859974,-0.01219867,-0.031126104,-0.0027307754,-0.034735575,0.04549714,-0.012744547,0.004812626,-0.029544177,0.014415598,0.014081387,-0.024486464,-0.0020428596,-0.004784775,0.03789943,-0.016276034,0.018225593,0.007948631,-0.018804891,-0.0067343344,-0.029833825,-0.009530559,0.024776112,-0.052894324,0.028118214,0.0037320133,0.017891383,0.008745166,0.0336661,0.011374285,0.0054504103,0.040729076,-0.009530559,-0.02936593,-0.0020038683,0.0053585027,-0.011274022,0.045185212,-0.014994895,-0.009012533,0.028140495,0.01677735,-0.0058208266,0.04407118,-0.025778743,0.004060653,0.003328176,0.0072579305,0.013412967,0.0032028472,0.015986385,-0.026781373,0.028831195,0.01155253,-0.034423646,-0.009508278,0.022648307,-0.024597866,-0.032730315,0.054008357,-0.021935325,-0.01449358,-0.015941823,0.012978494,0.009185209,0.0010569396,0.015596474,-0.0009692094,0.040016096,0.008979113,-0.05164661,-0.0017142196,-0.009753366,0.009686524,0.035493117,-0.0052526696,0.028185055,-0.045296613,0.016342876,-0.029744703,0.02967786,0.008138017,0.003258549,0.017701996,-0.012376916,-0.01744577,0.004250039,-0.002126412,-0.0034869257,-0.018804891,0.050265204,-0.030903298,-0.0067510447,0.003553768,0.0062553,-0.010276962,0.020275416,-0.017000156,0.028764352,-0.012878231,-0.00312765,0.024753831,-0.021032957,-0.03400031,-0.018593224,-0.030457685,0.020487081,-0.019640416,0.016755069,0.024597866,-0.0063499925,0.009430296,-0.0007902677,-0.042756617,0.028719792,0.019618135,0.028898036,0.00856692,0.0027293828,-0.0021431225,-0.02929909,0.01976296,0.0030524526,0.010639023,-0.011118057,0.043670125,0.02339471,0.0063555627,0.014861211,0.03587189,-0.023550674,0.0016264893,0.036785398,-0.020854713,0.00823828,0.030457685,-0.028073652,0.032173295,-0.003294755,0.023974007,0.0064446856,0.012365775,0.034356803,-0.013747177,-0.017624015,0.017646296,-0.0038796228,-0.0005267291,-0.0018855022,-0.036139257,0.018236734,-0.03752066,0.018593224,0.010678014,-0.012777967,-0.0057149935,0.0007888752,0.0045285476,0.05302801,-0.07432833,-0.014749807,-0.039080307,-0.013479809,-0.011574811,0.0056648618,-0.024865234,0.048883803,0.032396104,0.006812317,-0.02410769,0.0015693951,-0.0056453664,0.00016606066,0.048037138,-0.010505339,-0.010505339,0.025890145,0.0021848988,0.03649575,0.050265204,0.006494817,-0.0026207645,-0.045185212,0.02736067,0.009029244,-0.014014545,-0.026068391,0.004392078,-0.04616556,-0.013446388,0.0027753366,0.03324277,-0.0044394247,-0.008644902,-0.03433452,-0.029878387,0.017278664,-0.00954727,0.00989262,0.029722422,-0.0056509366,0.025177164,0.046566613,-0.010098716,-0.012677705,-0.02410769,-0.017557172,0.021823922,-0.0008215999,-0.041687146,0.017356647,0.009786787,0.030457685,0.041286092,0.009926042,-0.013379546,0.000051611096,0.031415753,0.0021835063,0.026246637,0.015618754,0.013880861,0.0049713757,-0.011184899,0.024620147,-0.01217639,0.00858363,-0.026291197,0.003857342,-0.02470927,0.015908403,0.0041609164,0.0029828255,0.0071353866,-0.02406313,-0.013791738,0.032819435,-0.0073971846,0.037832588,0.025065761,-0.016755069,0.006628501,0.030078912,-0.038122237,0.008310692,-0.013424108,-0.025979267,-0.023461552,-0.016342876,0.03495838,-0.024219096,0.022269536,-0.005015937,-0.013646914,-0.0409296,-0.017969366,0.0028004025,-0.03065821,0.017434629,-0.02539997,0.009018104,0.0072356495,0.029633299,0.00071576674,0.010031874,0.0075308685,-0.017334366,-0.0013208264,-0.00079444534,0.009474858,0.029477334,-0.0014273558,0.001399505,0.013468669,-0.0003241142,-0.00061237044,-0.014047966,0.02635804,-0.01811419,-0.008527929,0.014270773,-0.01217639,0.0015220487,-0.013947703,-0.0023130127,0.00027520116,0.012109548,0.00020818507,0.024486464,-0.016933315,0.04079592,0.004149776,-0.01946217,0.01414823,-0.016632525,-0.039815567,0.0041274955,-0.0009845274,-0.009118367,0.015518491,-0.011909021,0.022815412,-0.009313323,0.016175771,0.033710662,0.020888133,0.048170824,0.011023365,0.01910568,-0.00021131829,0.00510506,-0.011090207,-0.009279901,-0.0110122245,0.037832588,0.0059879315,-0.010639023,0.02733839,0.00988705,0.006400124,0.020230854,-0.009218629,-0.00024386897,0.017055858,-0.020008048,-0.018336996,0.006550519,0.0076979734,0.048571873,-0.0018743619,-0.012878231,0.03616154,-0.0051440513,0.01910568,-0.0028742072,0.007775956,0.00084318436,0.034445927,0.04315767,0.035002943,0.022291817,-0.007313632,0.0045898193,0.0066452115,-0.008806437,0.0054225596,0.019952346,-0.005798546,-0.01153025,-0.00477085,-0.040729076,-0.00049504876,0.0069070095,-0.019985767,-0.011374285,0.020810151,0.019807521,-0.061271857,0.010293673,0.0119647235,-0.014861211,0.030413123,-0.00955841,0.0014635619,-0.01053876,0.006895869,-0.019974627,0.009497138,0.021222344,0.032752592,0.011652794,-0.0013723504,0.003364382,0.00826056,0.012332355,0.029254528,0.024820672,-0.008160298,-0.0031248648,-0.014192791,0.02406313,0.019729538,0.041999076,0.017222963,-0.042957142,0.0065672295,0.052270465,-0.013546651,-0.0048516174,-0.0062107383,0.008466657,0.010120997,-0.008349683,0.016621385,-0.022146992,-0.0388575,-0.020386819,-0.041352935,0.007658982,-0.04438311,-0.013346125,0.0064224047,-0.0066452115,-0.021333747,-0.011251741,0.030836456,-0.012421477,0.0024090982,-0.008310692,-0.0068568783,-0.012978494,0.0020665326,0.009825778,0.029098563,0.037297852,0.05111187,-0.028318739,-0.012343494,-0.021901906,-0.015295684,0.019818662,0.0006558874,-0.004080149,-0.054855026,0.03099242,0.0052136783,-0.014415598,0.008371964,0.012142968,-0.00006162,-0.0067677554,-0.041642584,0.02504348,0.027494354,-0.012956213,-0.009441436,-0.002783692,-0.00034935403,0.018682348,0.025533656,-0.01811419,0.02273743,0.015351386,-0.035136625,-0.011402136,0.002501006,-0.047502402,-0.037052765,0.00957512,0.00009190779,-0.01812533,-0.014972614,-0.010672444,0.0036373204,-0.002637475,-0.021545414,-0.00239378,0.012699985,0.024508744,0.0042695347,0.015607614,-0.01252174,-0.0020707103,-0.0061494666,0.051869415,-0.01483893,-0.013424108,0.019740678,-0.010416216,0.00758657,-0.014326475,-0.003659601,-0.022146992,-0.007174378,0.00923534,0.015796999,0.026402602,0.03097014,-0.005397494,0.027182424,-0.0077258246,0.023751201,-0.010622312,-0.009357884,-0.06060344,-0.026424881,0.0036094696,0.0052526696,-0.042645212,-0.0077926666,-0.010639023,-0.023951726,0.0072022285,0.023684358,-0.013747177,-0.043714687,-0.014103668,0.005071639,-0.026959619,-0.004578679,-0.023639798,-0.0032084174,0.018916294,0.0012024603,0.015696736,-0.035938732,-0.017980505,-0.0027934397,0.037676625,-0.029499615,0.0057818354,0.0016473775,0.006416835,0.0056620766,-0.01776884,0.006494817,0.003094229,-0.007581,-0.010070866,-0.005670432,0.036852237,-0.03558224,0.014103668,-0.017880242,-0.037119605,-0.01715612,-0.0055061122,-0.020353397,0.011374285,-0.024174534,0.013591212,0.02441962,-0.039191708,0.00789293,0.03359926,0.0065393783,-0.018615505,-0.03417856,-0.041352935,-0.028095933,-0.019818662,-0.00019634845,-0.035136625,-0.022882255,-0.007842798,0.020219713,-0.006728764,0.04875012,-0.0052805203,0.008282841,-0.016120069,0.012432617,-0.010070866,0.03567136,-0.0103995055,0.01940647,0.051200993,-0.008795297,0.02473155,-0.0016613029,-0.011017795,-0.017590594,-0.02470927,0.0009037599,-0.004974161,0.013947703,0.025199445,-0.047591522,-0.023773482,0.011463407,0.0018492962,-0.012878231,-0.02078787,0.012878231,0.005770695,0.0125328805,-0.037386976,0.003294755,0.017612875,0.009068235,0.012544021,-0.020186292,0.03259663,0.020164011,0.011686214,0.017902523,0.00034413202,-0.00029643744,-0.009140647,0.038389605,-0.009363454,0.013268143,-0.013858581,0.005375213,0.034780137,0.025177164,-0.0026848214,-0.028853476,0.029031722,-0.013457528,-0.014894632,0.031215228,0.008394245,0.018994275,0.018593224,0.02076559,-0.012666564,0.005999072,-0.01646542,0.004392078,-0.01549621,0.025310848,-0.01681077,-0.003425654,-0.0042806747,-0.012332355,0.030390842,0.011251741,0.038010832,-0.009848059,-0.0015582548,0.019350767,0.0197741,-0.021133222,0.065326944,0.037943993,-0.011073496,-0.03723101,0.03226242,0.020509362,0.02566734,0.02666997,0.0015610398,0.01679963,-0.021333747,0.022503482,0.007937491,-0.014192791,0.010193409,-0.010254681,-0.03259663,0.025555935,0.023194185,0.0050883493,-0.00012419737,-0.005464336,0.014671825,-0.0028212906,0.029544177,-0.0072635002,-0.001176002,0.023595236,-0.014281914,0.021266906,0.0017323226,0.018582083,-0.046655737,0.020197432,-0.039102588,-0.0012783537,-0.03295312,0.000485301,0.0071298163,-0.005035433,0.004305741,0.009719945,-0.04079592,0.011190469,0.010995514,-0.0019161381,-0.03132663,-0.027739441,-0.0004964413,0.032485224,-0.027249267,0.00791521,-0.00626087,-0.004587034,0.016331736,-0.03433452,0.02441962,0.00954727,0.017746558,0.043068547,-0.0010820053,-0.031126104,0.072055705,0.008650472,0.0047290735,0.0015109084,-0.0066507817,0.026090672,0.00759214,-0.008973543,-0.023528393,-0.039035745,-0.044360828,-0.00958069,0.0016543402,0.014816649,0.0074584563,-0.0040996443,-0.0017782765,0.008873279,-0.012488319,0.01646542,0.025177164,0.03257435,-0.0059099495,-0.0107392855,0.01546279,-0.029187685,-0.035092063,-0.018515242,-0.00070706336,0.018437259,-0.022057869,0.004252824,-0.021913044,0.00691815,-0.018570943,0.01979638,-0.00064056943,-0.017568313,-0.020253135,0.009926042,-0.019317346],
        "Category": "Boutique",
        "Tags": [
            "pool",
            "free wifi",
            "air conditioning",
            "concierge"
        ],
        "ParkingIncluded": "false",
        "LastRenovationDate": "2019-02-18T00:00:00Z",
        "Rating": 3.60,
        "Address": {
            "StreetAddress": "140 University Town Center Dr",
            "City": "Sarasota",
            "StateProvince": "FL",
            "PostalCode": "34243",
            "Country": "USA"
        },
        "Location": {
            "type": "Point",
            "coordinates": [
                -82.452843,
                27.384417
            ]
        }
    },
    {
        "@search.action": "mergeOrUpload",
        "HotelId": "3",
        "HotelName": "Gastronomic Landscape Hotel",
        "Description": "The Gastronomic Hotel stands out for its culinary excellence under the management of William Dough, who advises on and oversees all of the Hotel’s restaurant services.",
        "DescriptionVector": [-0.048865054,-0.020307425,0.017633565,0.023875887,-0.04401433,-0.021689085,-0.04437217,0.011500583,0.03840817,0.00029058976,0.016907945,-0.009214383,-0.04512761,0.019889945,0.020973407,0.023040926,-0.026539808,0.050495215,0.07152826,-0.008786962,-0.009994673,-0.0053129313,-0.014601864,-0.048069853,0.021231845,0.022066806,-0.018021226,-0.010526463,0.07220418,0.0068685417,0.009472823,-0.023239726,0.040276892,0.03399481,0.0156058045,-0.001837658,-0.009567252,-0.03630089,0.009010613,0.027672967,-0.023398766,0.030078448,0.018428765,-0.006709502,-0.03598281,-0.018021226,-0.017782666,0.06655826,-0.019909825,0.010963823,-0.028428407,0.007325782,-0.030833889,-0.045724012,-0.0780489,0.024253607,0.018220024,-0.022762606,0.056777295,0.007817812,0.03355745,0.029163968,0.031967048,0.029959168,-0.051568735,0.057294175,-0.0156157445,0.03759309,-0.046002332,-0.020396886,0.053278416,0.016371185,0.03170861,-0.015685324,0.0010555041,0.024094567,0.0051886817,0.012872304,0.004055521,-0.03315985,-0.013568103,-0.023359006,-0.072243944,0.026480168,0.025068687,0.009010613,-0.018090805,-0.025207847,0.009408212,0.0025123358,0.024591567,-0.003725016,-0.0053924513,-0.025227727,-0.055385694,0.012136743,-0.011709323,-0.041310653,-0.021828245,0.04373601,0.030217608,0.023199966,-0.012912064,0.020277606,0.021609565,-0.031887528,0.014164504,-0.062264178,0.03315985,0.0034218458,-0.07550426,0.007653802,-0.04544569,-0.030973049,-0.0029298158,0.041708253,0.053198896,-0.03379601,-0.010834603,0.025168087,-0.031569447,-0.023836127,-0.025088567,-0.009935033,0.0017009829,-0.03395505,0.03174837,-0.030814009,-0.0155958645,-0.0030192758,0.009477792,-0.024830127,-0.046757773,0.0055216714,-0.015069044,0.024015047,0.015735025,-0.020655327,-0.020357126,0.015287724,0.003705136,-0.03389541,-0.026142208,-0.041390173,-0.03705633,0.06818842,0.03186765,0.007181652,-0.012802724,0.030694729,0.025366887,0.064729296,0.029680848,-0.011639743,-0.0016351305,0.0029944258,0.021788485,-0.017921826,-0.03486953,0.040992573,-0.021629445,0.03576413,-0.07232346,0.004868116,0.055783294,0.031112209,-0.046121612,-0.049262654,-0.04500833,-0.023021046,0.03538641,-0.020536046,0.006500762,0.031808008,0.03359721,0.052920576,-0.017812485,-0.014949764,0.028845888,0.019780606,0.019999286,-0.020874007,0.0865973,-0.057691775,0.019442646,0.03190741,-0.079122424,0.046519212,0.018170325,0.012196383,0.013448824,0.009865453,-0.0850069,0.0057204715,-0.03270261,0.051727775,-0.03242429,-0.041151613,0.012902124,0.0003308157,-0.011937943,0.0045102765,0.018617624,-0.016401004,-0.018369125,0.009716352,0.0052185017,-0.024850007,0.019880006,0.03294117,-0.004353721,-0.04373601,0.019134505,0.0693017,-0.016222084,-0.03570449,-0.050018094,0.003702651,-0.028448287,0.047791533,0.00023576444,0.0012723204,0.0047712014,0.028030807,-0.026162088,0.06846674,-0.0069281817,-0.025963288,-0.004067946,0.011848483,0.0010604741,-0.013090984,-0.024174087,-0.029541688,-0.014224144,0.04238417,0.007236322,0.0034392409,-0.03447193,-0.013001524,-0.03357733,0.007017642,-0.008697502,0.011450883,0.030058568,-0.019154385,-0.014104864,-0.022822246,-0.011013523,0.024631327,-0.0059391516,0.03238453,0.03644005,-0.028925408,0.020774607,-0.0029447258,0.0016053105,0.015426884,0.041946813,0.025426527,0.019094745,-0.000408472,0.056061614,-0.024492167,-0.012385244,-0.046996333,-0.054868814,0.030694729,0.00025517851,-0.059918337,-0.045843292,0.0029571508,-0.0068486617,-0.03745393,0.03638041,-0.031092329,0.0055167014,0.000035877198,-0.042145614,-0.0138861835,-0.022086686,-0.03785153,0.07232346,-0.013031344,-0.018657384,-0.006461002,-0.013826543,0.029422408,-0.023716846,0.007141892,-0.0025309732,0.0026788306,0.011659623,-0.03838829,-0.00011531956,-0.007922182,0.022881886,-0.06938122,0.002265078,-0.0021681632,-0.023736726,0.0750669,0.03610209,-0.014820544,-0.018041106,0.061429217,0.003287656,-0.029800128,0.020436646,0.022941526,-0.0022812306,0.020237846,-0.019184206,-0.0716873,-0.022066806,-0.039879292,0.014701264,-0.0058447216,-0.032245368,0.0060137017,0.010049343,-0.021470405,-0.0050147315,0.007718412,0.057413455,-0.023657206,0.011798783,0.025943408,-0.009199472,-0.0021818306,0.040952813,-0.032682728,0.018190205,-0.0026639206,0.022444526,0.016629625,-0.015466644,-0.014800664,0.024512047,0.0016475555,0.014512404,-0.058327936,-0.012653624,-0.010049343,0.064331695,-0.025983168,-0.010337603,-0.017971525,-0.013677443,-0.010993643,-0.056817055,-0.027593447,-0.009542403,0.010009583,0.014422944,0.014850364,0.007609072,0.054550733,-0.011073163,0.039839532,-0.024452407,-0.024929527,0.017822426,-0.007151832,0.014760904,0.007256202,-0.045724012,0.009646772,-0.027692847,0.017395005,-0.007678652,0.0056459215,0.013220204,0.009607012,-0.064013615,0.017116684,-0.001591643,0.008886362,-0.04234441,-0.041310653,-0.0020849155,-0.04294081,0.013478644,-0.028388647,-0.010526463,0.022265606,-0.004798536,-0.014870244,-0.027573567,0.057015855,0.04492881,-0.011560223,0.0049749715,-0.008364513,-0.011540343,0.010228263,0.015029284,0.052960336,-0.021331245,-0.0029397558,0.058407456,-0.04341793,-0.011083103,-0.022524046,0.03178813,-0.014661504,-0.03381589,-0.055226654,-0.0069679418,0.0030714609,-0.03801057,-0.023796367,0.008453973,-0.019015225,0.024273487,0.007400332,0.04262273,-0.026818128,-0.0122659635,0.004773686,0.047155373,0.03572437,0.03767261,0.03482977,0.012941884,0.018597744,0.0012580316,-0.012216263,-0.07781034,-0.03226525,0.015019344,-0.0059987917,0.023120446,0.057135135,-0.019989345,0.060196657,-0.020416766,0.012434944,-0.024392767,-0.021072807,-0.057612255,-0.016659444,-0.04242393,-0.03741417,0.023040926,0.051250655,0.012434944,0.062184658,0.011480703,0.0044829412,0.0021992256,-0.017723026,-0.020327305,0.021132445,-0.03870637,0.07005714,0.019383006,-0.016540164,0.03753345,-0.03630089,0.018786605,-0.010755083,-0.022186086,-0.0048109614,0.052284416,0.023418646,-0.011997583,0.017563986,0.024671087,-0.013130744,-0.04520713,-0.030376649,0.04298057,0.04266249,0.018667325,-0.03809009,-0.029621208,0.053397696,0.041787773,-0.018776665,0.015347364,-0.004224501,-0.03628101,-0.030654969,-0.024333127,-0.004423301,-0.050813295,0.021689085,-0.002314778,0.016748905,0.008260142,0.020476406,0.052801296,-0.004714046,0.015814545,-0.0106358025,-0.049103614,-0.027513927,-0.030913409,-0.016301604,-0.008329722,0.040833533,0.03230501,-0.050614495,0.014492524,0.03178813,-0.011361423,-0.024472287,0.017375125,-0.018895945,-0.032245368,0.017544106,-0.017315485,0.010437003,-0.024333127,0.020317366,0.010685503,0.012484644,0.011629803,-0.015874185,0.009602043,0.001994213,-0.023895767,0.030575449,0.036121972,0.039680492,-0.03906421,-0.028726608,0.026818128,0.027275367,-0.018548045,-0.03502857,-0.014949764,0.03822925,0.010347543,0.0016599805,-0.009626892,-0.021708965,0.011669563,0.0053328113,0.060554497,0.003690226,-0.0015096379,-0.030257368,-0.026261488,-0.0060137017,-0.03385565,-0.010054313,0.024909647,-0.024154207,0.011232203,0.0012617591,0.008881393,0.03846781,0.014939824,0.009318752,0.048030093,-0.016351305,-0.014234084,0.019621566,-0.055504974,0.023677086,-0.021013167,0.001827718,0.03508821,0.003188256,0.025585568,0.04262273,0.0028428407,-0.026639208,0.023319246,-0.014353364,-0.014661504,-0.003752351,0.0155958645,0.010526463,0.012096983,0.010745143,0.013498524,0.049739774,0.03357733,-0.016162444,0.023279486,0.021152325,-0.04298057,0.013975644,0.018329365,0.025187967,-0.017563986,0.03592317,0.006396392,-0.007166742,0.04512761,0.0057055615,-0.010675563,0.006267172,-0.04302033,-0.0021656782,-0.03371649,0.026838008,-0.028567567,0.0015270329,0.003747381,-0.011102983,0.004674286,0.025963288,-0.008419182,0.023100566,0.010397243,-0.025923528,0.012723204,0.0028030807,0.022782486,0.011699383,-0.029979048,0.03618161,-0.0041524363,-0.007822782,0.019363126,-0.026678968,-0.048825294,0.041787773,0.03250381,-0.0009753628,0.0054620313,0.017086865,-0.011679503,-0.022245726,-0.010153713,-0.029641088,0.013289784,0.023696966,0.048109613,-0.003876601,-0.0017991405,0.020714967,0.03528701,-0.027494047,-0.017822426,-0.000420897,-0.018418824,0.024531927,0.040634733,0.028150087,-0.010755083,-0.008439062,0.04461073,0.007499732,-0.025863888,0.030555569,-0.028746488,0.0027633207,-0.0028080507,-0.008106072,-0.0053924513,0.0058099316,0.031450167,0.023498166,0.011490643,-0.003946181,0.007385422,0.026639208,-0.04234441,0.0034119058,-0.047274653,0.0052980212,0.0026142208,0.025207847,-0.010755083,0.012762964,0.031132089,0.012474704,0.022265606,0.04282153,0.007355602,0.0097561125,-0.016609745,-0.058009855,-0.020893887,-0.029959168,-0.000963559,0.017255845,-0.020635447,-0.009129892,0.023597566,-0.023677086,-0.011659623,0.026420528,0.024929527,0.0015096379,0.007286022,0.016013345,0.029342888,-0.012494584,-0.015387124,-0.046638492,0.0005808689,-0.009020553,-0.010506583,-0.0022961407,-0.0017320454,-0.019720966,0.015188324,-0.020734847,0.001775533,-0.0022439556,0.029919408,-0.003777201,0.06484858,-0.003720046,-0.008150802,-0.015516344,0.013786783,-0.06635946,0.007117042,-0.0049724863,0.00073990895,0.011102983,-0.020098686,-0.0044879112,-0.017116684,0.028547687,0.0058149016,-0.012603924,-0.03693705,-0.020317366,-0.0024029957,-0.020933647,-0.024392767,-0.003772231,0.030654969,0.010456883,-0.030873649,-0.03606233,-0.008230322,-0.000918829,-0.023696966,0.001904753,-0.04512761,-0.013438884,-0.025585568,-0.029183848,-0.03375625,0.010327663,0.03242429,-0.009050372,0.008160742,-0.03427313,-0.049183134,0.029422408,-0.021987285,0.0025819158,-0.011262023,0.014959704,-0.0020724905,-0.030953169,0.03453157,-0.0027384707,-0.011331603,0.060156897,0.008339662,0.00091448025,-0.039322652,-0.0013742053,0.03226525,-0.03321949,-0.002004153,0.06556426,-0.008946002,-0.023875887,0.0012238629,0.0013829028,0.013717203,-0.022961406,0.026241608,-0.024452407,-0.028189847,-0.020039046,-0.0018836305,-0.021430645,0.0052532917,-0.018488405,0.011301783,-0.0059292116,0.018319424,0.008066312,-0.0002029935,-0.024730727,0.010745143,0.023836127,0.022365006,-0.018518224,0.022702966,-0.007350632,-0.0015531254,-0.014025344,0.015407004,-0.017275725,-0.0014524829,0.006441122,0.003921331,0.0054570613,-0.010715323,-0.032483928,-0.0026564656,0.03584365,-0.0061429217,0.012951824,0.030992929,-0.022504166,0.00093436026,0.017335365,0.04508785,0.03492917,0.007131952,-0.0028826008,-0.014442824,0.018269725,0.021271605,-0.016053105,-0.020108625,-0.029760368,-0.0052334117,0.040833533,0.002319748,0.0068337517,-0.014780784,0.006769142,0.0054471213,-0.00095424027,-0.03910397,0.023975287,-0.015198264,0.016381124,0.022702966,-0.025923528,0.002024033,-0.009621923,-0.004868116,-0.019899886,0.019522166,0.028329007,-0.052284416,-0.0010331391,-0.027474167,0.014830484,0.008906242,0.03604245,-0.023557806,0.023359006,0.0021731332,-0.008488762,0.03659909,-0.019750785,-0.016361244,0.011729203,-0.026559688,-0.015814545,0.010516523,-0.018180264,-0.065047376,0.011550283,0.007619012,-0.07443074,-0.006709502,0.03789129,-0.007305902,-0.027692847,-0.008583193,0.014949764,-0.032006808,0.020078806,0.061667778,0.014015404,0.021788485,0.0029571508,-0.0005147058,0.0044307564,-0.015645564,-0.07121018,0.007355602,-0.014413004,0.03286165,0.015804604,-0.0015481554,0.030674849,-0.03640029,0.0097561125,-0.017673325,-0.013796723,0.03387553,0.019790545,0.026022928,0.009010613,-0.025744608,-0.0002453938,-0.022842126,0.058447216,-0.03719549,0.027633207,-0.006774112,-0.012146683,0.016768785,-0.0045127613,0.009462883,0.039362412,-0.03238453,-0.004000851,0.020416766,0.018130565,0.0082551725,0.003792111,-0.008220382,-0.0090354625,0.016033225,-0.00014125675,-0.0026092508,0.015715145,-0.0044009364,-0.018776665,-0.011202383,0.019522166,-0.052841056,-0.00079892774,0.041668493,0.03737441,-0.013230144,0.021132445,0.004219531,-0.04278177,-0.018886006,-0.021251725,0.024154207,-0.009656712,0.04274201,0.014611804,0.001763108,0.0098058125,-0.0060236417,-0.0017519254,0.010864423,0.020675207,-0.021033047,0.003993396,0.014452764,-0.03993893,0.025466288,0.022524046,-0.008598102,0.000807004,-0.0023185057,0.017295605,-0.012574104,-0.023796367,0.012822604,0.025943408,-0.004463061,0.0006352283,0.008439062,-0.008886362,-0.03353757,0.022563806,-0.000978469,-0.0009703928,0.0013207778,-0.00025067443,-0.047513213,0.06437146,0.010407183,0.0008132165,-0.0051290416,0.029959168,-0.0090652825,-0.001760623,0.019035105,0.04329865,0.021887885,0.025227727,0.0029273308,0.030654969,-0.024810247,-0.022325246,0.024551807,-0.028229607,0.031291127,0.029303128,0.03347793,0.020595687,0.022643326,0.030615209,0.03379601,0.006272142,0.016152505,0.0015730055,-0.03451169,0.019293545,0.009159712,-0.0017071954,-0.011431003,0.03816961,-0.027454287,0.024850007,-0.016798604,0.019333305,-0.0047314414,0.03447193,-0.006421242,0.023557806,0.029004928,-0.022126446,0.029124208,0.016639564,0.023060806,0.009527492,-0.047234893,0.008667682,0.029521808,0.014104864,-0.008444033,0.019273665,0.022484286,-0.012504524,0.030118208,0.0024005107,0.0024676058,-0.0082054725,0.026142208,0.010198443,-0.001658738,0.021172205,0.023995167,-0.009701443,0.0025471258,0.03282189,0.0058049615,0.0027956257,0.028428407,-0.0034243308,-0.0047960514,0.024193967,-0.0139160035,0.012802724,-0.008369482,-0.011311723,-0.031768247,0.032762248,0.0012226204,0.019780606,0.010496643,0.006510702,-0.009244203,-0.0060385517,-0.007748232,-0.015844364,-0.020834247,0.0068586017,0.012057223,-0.028309127,0.009000673,-0.0058745416,0.015665444,0.007161772,0.024969287,-0.040754013,-0.024034927,0.012464764,-0.041072093,0.0082154125,-0.014711204,0.040754013,-0.006441122,0.015377184,-0.03797081,-0.024571687,0.03170861,-0.017285665,-0.026122328,-0.0013307178,-0.010794843,0.006674712,-0.004239411,0.052204896,-0.023975287,-0.0023011107,-0.03347793,0.03162909,0.03651957,-0.024512047,0.016560044,0.007763142,0.003367176,0.018647445,-0.03323937,0.0077929622,0.015695265,0.004028186,0.011083103,0.016092865,-0.014263904,0.030953169,0.002411693,-0.013627743,-0.017096804,-0.0105960425,-0.031151969,0.008752173,0.007499732,-0.03703645,-0.04524689,-0.017454645,-0.009318752,0.016331425,0.017941706,0.052761536,0.011331603,0.058168896,0.049103614,0.022424646,0.03250381,0.009537432,-0.010934003,-0.021569805,0.007236322,0.018895945,-0.0015779755,-0.020993287,-0.008970853,-0.029462168,0.008150802,0.012842484,0.013200324,0.00045506575,-0.011778903,0.011540343,0.003225531,0.027752487,-0.009522523,0.013727143,0.014681384,0.010516523,-0.007867512,0.017216085,-0.0047910814,0.003976001,0.03735453,-0.025068687,0.04361673,0.04321913,0.029541688,-0.0015208204,0.0051290416,-0.0010256841,0.026500048,-0.00057931576,0.023160206,0.0090354625,-0.017454645,-0.04345769,-0.001827718,-0.023498166,-0.008125952,0.006217472,0.025486168,-0.019432705,0.010407183,0.032006808,-0.03250381,0.019373065,0.007107102,-0.006585252,0.049183134,0.023597566,0.026460288,0.021271605,0.019263726,0.012087043,0.010864423,0.029601328,0.006490822,0.006341722,-0.00012300753,-0.0009902727,0.020476406,-0.03453157,0.026539808,-0.023796367,-0.0057950215,-0.003570946,-0.013776843,0.00049513637,0.001666193,0.047195133,0.020377006,-0.023875887,0.003623131,0.025724728,-0.029482048,-0.007156802,-0.012832544,-0.016609745,-0.011212323,0.049501214,-0.010034433,-0.010456883,-0.013329544,0.017335365,-0.023438526,0.008657742,-0.007723382,-0.006490822,0.0023359007,0.0059640016,-0.0138762435,-0.0053775413,0.03479001,-0.029243488,-0.011609923,-0.006242322,0.00025347006,-0.04294081,-0.025187967,0.029482048,0.03759309,0.013587983,0.018687205,-0.03371649,0.017275725,0.000058242204,0.007102132,0.007276082,-0.009790903,-0.026122328,-0.03310021,0.014174444,0.019482406,0.030018808,0.013458764,0.03622137,-0.029641088,0.017166385,-0.011371363,0.03914373,0.026360888,0.018995345,-0.006366572,-0.027036808,-0.015913945,0.023776487,0.006719442,-0.018150445,0.007271112,-0.012882244,-0.017295605,-0.026102448,0.015735025,-0.003754836,0.0048258714,-0.022404766,-0.040913053,0.016251905,-0.030217608,-0.028309127,-0.04397457,-0.03504845,-0.012395184,-0.039402172,-0.019542046,0.03584365,-0.020555926,0.0053029913,-0.030217608,0.013170504,-0.017424826,0.04357697,0.008439062,0.012385244,0.021410765,0.040594973,0.009989703,0.021748725,0.041429933,-0.019204086,-0.013737083,-0.0055564614,0.025068687,0.04286129,0.00016866942,0.025048807,-0.014234084,0.0015904005,-0.024472287,-0.003556036,-0.0011983915,-0.029541688,-0.030237488,0.012444884,-0.039760012,0.000839309,0.022365006,0.028786248,-0.03315985,0.025744608,-0.007067342,-0.012643684,-0.026102448,0.001646313,0.019243846,-0.023875887,-0.010735203,-0.017474525,0.0077929622,0.04425289,-0.026181968,-0.017305546,-0.018687205,0.032086328,0.009070252,0.017872125,0.003565976,0.0025844008,0.011599983,0.027692847,0.023140326,-0.004130071,0.03496893,0.018051045,-0.03437253,-0.011122863,0.0053825113,-0.012534344,0.007837692,0.030515809,0.006207532,-0.03292129,-0.011252083,0.009830663,-0.010228263,0.013468704,-0.011878303,-0.008125952,0.04317937,-0.04536617,0.003700166,-0.016440764,0.020635447,0.0038964811,-0.028070567,-0.015049164,-0.0051340116,-0.0041027362,-0.004299051,-0.054630253,-0.0048879962,0.0139259435,0.017116684,0.010387303,0.012872304,-0.019780606,-0.0017892005,0.009020553,0.010218323,0.032642968,0.004818416,-0.026380768,0.0058248416,0.040952813,-0.021828245,-0.014452764,0.0011344028,-0.03194717,0.007857572,-0.012067163,-0.0047264714,0.0022153782,-0.0030764309,-0.021052927,-0.015983524,0.00041685888,-0.022464406,0.0089957025,-0.03974013,0.0022663206,0.052284416,-0.020436646,0.026758488,-0.0027484107,0.0056757415,-0.0058795116,0.011093043,0.023199966,-0.00015049786,0.012444884,0.03296105,-0.010536403,-0.011321663,-0.020029105,0.007991762,-0.041867293,0.025863888,-0.003715076,0.016490465,0.014333484,0.0034566359,0.029998928,-0.003106251,-0.006257232,-0.027613327,-0.023239726,-0.013230144,-0.054113377,0.03447193,-0.008001702,0.012037343,0.009597072,0.017554045,-0.0021905282,-0.001618978,0.0082651125,-0.0011232203,-0.007927152,-0.018478464,-0.009974793,-0.019303486,-0.039600972,-0.0155859245,0.030476049,0.017226025,-0.024313247,0.010039403,0.004234441,-0.018707085,0.011023463,-0.018488405,0.016798604,0.04373601,-0.024710847,0.0069281817,0.0028279307,0.004015761,-0.009085163,0.0030590359,-0.0059242416,-0.010705383,-0.012653624,0.000961074,0.011142743,-0.003352266,0.047871053,-0.04544569,-0.0015233054,0.023875887,0.00043021573,0.007474882,-0.017206145,-0.004443181,-0.014035284,0.018001346,0.0011685715,-0.020635447,0.0014760904,0.027832007,0.015397064,-0.031489927,0.015327484,0.014015404,0.013518404,0.028905528,-0.024690967,0.024810247,-0.026181968,-0.015317544,0.010993643,0.011450883,0.030297128,0.009920123,-0.020068865,0.0053029913,0.015108804,0.0007604102,-0.019144446,-0.017365186,0.003215591,0.0025148208,-0.023975287,0.004709076,-0.021390885,-0.012027403,0.024869887,-0.028905528,-0.03638041,0.0043015364,-0.029859768,0.006232382,-0.0069629718,-0.0024464831,0.013120804,0.031231489,0.0082154125,0.0013232628,0.0028552657,-0.009219352,0.019144446,-0.006774112,0.0034094208,-0.03500869,0.026221728,-0.026718728,0.03735453,0.0155958645,-0.0059242416,-0.023677086,-0.014691324,0.013359364,-0.004299051,0.016500404,0.009209412,0.004085341,-0.010774963,-0.004738896,0.016490465,-0.027315127,0.021033047,-0.03445205,0.031092329,0.006749262,-0.008806842,-0.008056372,-0.04441193,0.014184384,-0.0155859245,0.016560044,0.007375482,0.011440943,-0.026162088,-0.018120624,-0.012772904],
        "Category": "Suite",
        "Tags": [
            "restaurant",
            "bar",
            "continental breakfast"
        ],
        "ParkingIncluded": "true",
        "LastRenovationDate": "2015-09-20T00:00:00Z",
        "Rating": 4.80,
        "Address": {
            "StreetAddress": "3393 Peachtree Rd",
            "City": "Atlanta",
            "StateProvince": "GA",
            "PostalCode": "30326",
            "Country": "USA"
        },
        "Location": {
            "type": "Point",
            "coordinates": [
                33.847848,
                -84.363602
            ]
        }
    },
    {
        "@search.action": "mergeOrUpload",
        "HotelId": "4",
        "HotelName": "Sublime Palace Hotel",
        "Description": "Sublime Palace Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 19th century resort, updated for every modern convenience.",
        "DescriptionVector": [-0.05908324,0.007784768,0.027741672,-0.007171661,-0.027066693,-0.017335733,-0.014467069,-0.03316401,0.0033411505,-0.017594475,0.0030992825,0.00602982,-0.018033212,-0.02180185,-0.0069016693,0.05395339,-0.0099728275,0.023466801,0.05804827,0.06452808,-0.02112687,-0.046708606,-0.012149638,-0.011530906,-0.007773518,0.025784232,-0.045516144,-0.027269186,0.054538373,0.027021695,0.0035492692,-0.024434272,0.06308812,0.011137168,0.019023184,0.0043283086,0.00014492733,-0.013240855,0.0066485517,-0.0035548941,-0.012633373,-0.026864199,-0.001743698,-0.008364126,-0.02483926,0.023646794,-0.016593255,0.06268313,-0.0013612094,0.03617892,0.012138388,-0.011868396,-0.02920413,-0.03386149,-0.02951912,-0.011542156,0.016615754,-0.041376267,0.029789113,0.00015046426,0.024254277,-0.007014166,0.004055504,0.03660641,-0.067093,0.036696404,-0.049903512,0.023601796,0.0068566706,-0.015918275,0.019428171,0.0022274335,-0.01409583,0.0017633849,0.026909199,0.0051185973,0.029766612,0.034581468,-0.012228386,-0.042343736,-0.042096246,0.0267967,-0.00058568566,0.08225755,-0.02040689,0.0015960461,-0.04733859,0.03982381,-0.032691527,0.031701554,0.06808297,-0.01509705,-0.055573344,-0.01741448,-0.034401473,-0.02987911,0.012127139,-0.0035323948,-0.0368314,0.0536834,0.036741406,-0.09971703,-0.0013823025,-0.04958852,0.015142049,0.013128359,0.048418555,0.0013963646,0.01040594,-0.03660641,-0.09305722,0.01273462,-0.04517865,0.008679116,-0.004381744,0.010186572,0.03327651,0.004845793,-0.015592035,0.013848337,-0.021565607,0.008515996,-0.023241807,-0.0137470905,0.013297103,-0.046933603,0.038698845,-0.058813248,-0.016222017,-0.044661168,-0.010642183,-0.030351596,-0.043783695,-0.015209546,0.02618922,0.036291417,0.00054314785,-0.012768369,-0.03219654,-0.039036337,-0.0035633312,0.0038558226,-0.032691527,0.0069860416,-0.025289247,0.03552644,-0.02751668,-0.013218356,-0.036403913,0.047563583,0.013915835,0.03863135,-0.017943215,-0.008487872,-0.056518316,-0.023916787,0.01573828,-0.035076454,-0.021621855,0.056788307,-0.046843603,-0.019158179,-0.04088128,-0.030014105,0.011289039,0.0141408285,-0.045763634,0.022713073,-0.056833304,-0.014005832,0.037416384,-0.0040639415,0.005188908,0.041646257,0.025986725,-0.00163542,0.029001635,-0.0070647895,-0.020103151,0.02411928,0.0058160764,-0.023399303,0.08320252,0.006839796,0.040498793,0.0013471473,-0.022094341,0.019551918,-0.01407333,0.037776373,0.012464629,-0.029766612,-0.040431295,0.032309037,-0.01943942,0.011902145,-0.036066424,0.047788575,-0.02848415,-0.027561678,-0.042793725,-0.016075771,0.040431295,0.0017858843,0.004474554,-0.047923572,0.04832856,0.014107079,-0.015569536,0.017391982,-0.011401535,0.003405836,-0.009505967,0.022420581,0.028574148,0.00089223904,-0.008386625,-0.009460968,0.0071097882,0.034671467,-0.023939285,-0.010372191,0.014489568,0.02985661,0.0023539923,0.03311901,0.0067272997,-0.015704531,0.00804351,-0.009775959,0.026661705,-0.028011665,-0.035863932,-0.0045983004,-0.032309037,-0.0044323676,0.015805779,-0.01056906,0.022915566,-0.00073966547,0.028056664,-0.005388589,0.015817028,0.005020163,0.03226404,-0.0155020375,0.028281657,-0.007272908,0.0026042974,0.033096515,-0.014905806,0.024636766,0.015569536,-0.0301491,0.0103384415,0.014467069,-0.033411503,-0.03959882,-0.00071540836,-0.020913126,0.03890134,0.000020005507,-0.011182167,0.00074177474,0.022656824,-0.03719139,-0.0085047465,0.024929257,-0.020508138,-0.023511799,-0.033726495,-0.011924645,0.020204397,-0.03226404,0.04124127,-0.014455819,0.0041398765,-0.021329364,-0.023376804,0.032016546,-0.021385612,-0.032646526,0.0032877144,0.00095059664,-0.027269186,0.0027083568,0.03185905,0.008060385,-0.011120293,0.014467069,0.021104371,-0.002588829,0.0066541764,-0.011564655,-0.067093,-0.010096574,-0.0135895945,-0.019968154,0.0018618195,0.030171601,0.0015651096,-0.000103356295,0.038856342,0.026751703,-0.00072208786,0.01878694,-0.01777447,-0.03485146,-0.015400791,0.004840168,0.014107079,-0.013510847,-0.0045392397,-0.047383588,-0.018291954,-0.024996756,-0.021363113,0.023601796,-0.016615754,-0.048823543,0.093147226,0.02046314,0.0029305376,0.026211718,0.012149638,-0.013364602,-0.0010926237,0.0072672833,-0.006918544,-0.02411928,0.03318651,-0.035661437,0.034288976,0.0021810287,0.015490788,0.00094567495,-0.010782803,0.035053954,0.031094072,0.024884257,-0.031994045,-0.03156656,-0.00318928,0.013229606,0.022341834,-0.013882086,-0.054223385,0.0068172966,0.06529305,0.013207106,-0.06686801,-0.06308812,-0.036403913,-0.018280705,0.031206569,0.032331537,0.03455897,0.020744381,-0.04661861,0.023534298,-0.0057007675,-0.009635338,0.0064910566,0.033703994,0.028619146,0.014815808,-0.029316626,0.028371654,-0.00905598,0.028821642,-0.072627835,-0.027854169,0.0012810555,0.006766673,-0.015693283,0.023939285,0.04227624,-0.015367042,-0.036021426,-0.039216332,0.018854437,-0.03887884,0.04333371,0.0028700707,0.058228265,-0.034153983,0.035053954,-0.03262403,-0.036696404,0.023624295,-0.0036730154,-0.038811345,0.011401535,-0.0117558995,0.027921667,0.009162852,0.037078895,0.031049075,-0.015592035,-0.010647807,0.015828278,-0.036088925,-0.0031639682,-0.073032826,0.0027392933,-0.05431338,-0.026549209,-0.060208205,-0.023309305,-0.011969643,-0.03419898,-0.03419898,0.0234893,0.0048289187,-0.012903365,-0.0018772878,-0.024411771,-0.02245433,0.0704229,-0.0012648841,0.055618342,-0.007846641,0.045831133,0.011789649,0.033636495,0.05098348,-0.00005387535,-0.066103026,-0.027786672,-0.015018302,0.013882086,-0.04796857,0.036696404,0.0006243564,-0.016840748,0.050848484,-0.025064252,-0.002796948,-0.03998131,-0.02886664,-0.030306596,-0.058273263,-0.06403309,0.016638255,-0.0013970677,0.05161346,0.030711584,0.031724054,-0.02276932,-0.027561678,-0.021228118,0.0054645245,-0.012543376,-0.004603925,0.033231508,0.018426951,0.039306328,-0.032286536,0.020418141,0.005022975,0.057283293,-0.010366566,-0.007863516,0.020361893,0.027944166,-0.028101662,-0.009927829,0.05075849,0.0017957278,-0.038158864,-0.013364602,-0.027359184,0.03212904,0.0067272997,-0.011491532,-0.072987825,-0.01644701,0.039351325,0.023331804,0.022893067,-0.040003806,-0.025986725,-0.0006162707,-0.004345183,-0.02077813,-0.03795637,-0.0040920656,0.03883384,-0.07978262,-0.030126601,-0.0035155201,-0.0037405135,-0.0014322229,-0.008589119,-0.007278533,0.064303085,-0.061153177,0.0027491369,-0.014692062,0.020249397,0.014703312,0.01205964,-0.040363796,0.008009762,0.027359184,0.014444569,-0.015614535,0.019540668,0.02479426,-0.03154406,0.024749262,-0.020620635,0.024096781,-0.008296628,0.016469508,0.011305913,-0.022341834,-0.015052051,0.0108953,-0.015727032,0.023736792,-0.004927353,-0.0056023328,0.0030430343,0.0032905268,0.06295312,-0.041668758,-0.0184832,0.019585665,-0.011879646,-0.020710632,-0.07073789,-0.02515425,0.044661168,-0.001587609,0.015862027,0.023196809,0.010737805,0.023061812,-0.018865688,0.013915835,0.032151543,0.008251629,-0.008639743,-0.00871849,-0.015637035,0.009865955,-0.005183283,0.019900657,0.038721345,0.043221213,0.013252105,0.017718222,-0.015524537,-0.011902145,-0.01709949,0.009770334,0.040656287,-0.027899168,0.016379511,0.0020122838,-0.0022668075,0.021149369,0.0009787208,0.016233265,0.015850777,0.02283682,0.036853902,0.009832207,0.014557066,0.012014641,0.013634593,-0.0094497185,0.016537007,-0.014152078,-0.01816821,0.005664206,-0.051793456,-0.017200736,-0.0009618463,0.007914139,-0.010647807,0.026279217,-0.009235974,-0.023241807,-0.02077813,0.015052051,0.011699651,-0.005683893,0.03390649,0.01088405,0.022656824,0.020969374,-0.028461652,0.024501769,0.015445789,-0.023241807,-0.007076039,-0.039013837,-0.002074157,-0.02147561,0.0041258144,-0.0054898364,0.0027603866,-0.002393366,0.031994045,-0.009359721,0.027111692,0.006344811,-0.01984441,0.013859587,-0.005568584,0.014905806,0.015355792,-0.05858825,0.0034086483,0.0010933268,-0.026009224,-0.013634593,-0.016390761,-0.00920785,0.004547677,0.0039992556,0.013803339,0.0006879873,0.050353497,0.011401535,-0.04452617,-0.034288976,-0.0028335094,0.019068182,0.044571172,0.026751703,-0.030104103,-0.013240855,0.0058554504,0.036583908,-0.019326923,-0.0038501977,0.0005192423,-0.004210187,0.025649235,0.015648283,-0.013634593,0.0060579446,0.015862027,0.026549209,-0.019270675,0.048283562,-0.031994045,-0.011857146,0.024749262,0.00888161,-0.0066598016,-0.015727032,0.024884257,-0.012892116,0.01943942,0.00687917,0.020823129,-0.036943898,-0.001269806,-0.008949108,0.004677048,0.001615733,-0.015670782,0.015884526,-0.010349692,-0.016998243,0.014455819,0.068982944,0.012554626,0.011733401,0.022094341,-0.000056380155,0.05264843,0.0060916934,-0.07001791,-0.0047361087,0.0038080115,0.004952665,0.027089192,-0.03525645,-0.001378787,-0.0038980087,-0.007711645,-0.0065585542,0.031949047,-0.009517216,-0.020834379,0.011469033,-0.0063841846,0.020496888,-0.0042608106,-0.00770602,-0.007886015,-0.032669026,-0.032804023,0.0070647895,-0.026234217,0.008397874,-0.022904318,0.022589326,-0.035728935,0.0010891082,-0.021531858,-0.0054757744,-0.04292872,0.013240855,-0.022578077,0.020069402,-0.00006429008,0.008150382,-0.016357012,-0.017043242,0.031206569,-0.036313917,0.02076688,0.023106812,-0.038743846,-0.00071927544,0.04832856,-0.009151602,0.0055517093,-0.014995803,0.0022316521,-0.027989166,-0.020530637,0.015670782,0.07478777,0.0071941605,-0.01641326,-0.013454599,-0.008470997,0.0036589534,-0.012779619,-0.03426648,0.034963958,-0.0035689562,-0.014568316,0.007436028,-0.021565607,-0.021340614,-0.0028742894,0.03284902,-0.011609654,-0.015468289,-0.024569267,-0.049903512,0.031004075,-0.010979673,0.0063223117,0.018370703,-0.0011706682,0.03527895,-0.03485146,0.016885746,0.0071885358,-0.013533346,0.027044194,0.002027752,-0.006069194,0.0025592986,0.046573613,0.028934138,-0.0016986993,-0.021531858,0.03896884,-0.010141573,-0.029429123,-0.033748996,-0.011767149,-0.031206569,0.006294187,0.02884414,-0.015468289,-0.0060579446,-0.020688133,-0.0043901815,0.001385115,0.034738965,0.0075260256,0.029789113,-0.022195589,0.0011390286,0.03818136,0.0031724055,-0.026954196,0.031139072,0.028056664,0.03246653,0.003718014,-0.01240838,-0.024996756,0.01642451,0.005967947,0.02720169,-0.061603162,0.0056867055,0.019585665,0.026571708,0.005776703,0.00906723,-0.03554894,-0.00887036,0.038766343,-0.040993776,0.02280307,0.055798337,0.0024453958,0.014129579,-0.0006855264,-0.0033608372,-0.0022091528,0.028731644,-0.023939285,0.004536427,0.0009745022,-0.010321567,-0.010332817,-0.029316626,-0.012025892,0.014579565,0.010214696,0.0033242758,-0.012565875,-0.008594744,0.020204397,-0.013578345,-0.05188345,-0.042073745,-0.015187047,-0.008769114,0.029001635,-0.017009493,0.014455819,-0.021340614,0.032399032,0.010951549,0.014827058,0.0026872635,0.038653847,-0.016840748,0.008954733,0.024569267,-0.039711315,0.026729204,-0.014647063,-0.015198297,0.02954162,0.031409062,0.0017886966,0.037731376,0.0043367455,-0.013702092,0.026346715,0.005804827,0.011969643,0.0052873422,-0.029654115,-0.054178383,0.0018561947,0.0026057037,-0.023579298,-0.039846312,0.03883384,0.008527245,-0.019315675,0.008487872,-0.0049020415,-0.047158595,0.014973303,0.0012016048,-0.017560726,0.013454599,0.01709949,0.005804827,0.037708875,0.006440433,-0.043446206,-0.007076039,-0.020620635,-0.0020825942,0.02983411,0.011778398,0.0078072674,-0.021745602,0.049273532,-0.0051270346,-0.014287074,0.0155020375,-0.022735571,0.0029164755,0.030689085,-0.03755138,0.045493644,0.013128359,0.031724054,-0.018078212,0.00022024734,-0.030036604,-0.002812416,0.007481027,-0.03422148,0.030599087,0.03660641,-0.022263085,0.012093389,0.02879914,-0.004778295,0.01274587,-0.022566827,-0.025761733,0.015963275,-0.0012782431,0.008133507,0.00285179,0.010630933,-0.007492277,0.028349156,-0.0335465,0.017594475,-0.01122154,0.0060354453,0.043356206,-0.004755796,-0.019034432,0.04092628,0.026616706,-0.03282652,0.0027055442,0.0109178,-0.007756644,0.011542156,0.03991381,0.026954196,0.025986725,0.028731644,0.04468367,0.017155739,0.04661861,-0.0014350354,-0.0068341712,0.020721883,0.01644701,-0.04868855,0.029901609,0.011272164,0.013038361,-0.020305645,0.007239159,0.0058498257,-0.010147197,-0.0034367726,-0.026459211,0.016390761,-0.03788887,-0.0139045855,-0.022296835,-0.00077411754,-0.012914615,-0.0267967,-0.00887036,-0.00007852793,-0.017043242,-0.0043620574,0.006710425,0.028641647,-0.039396327,0.058768246,-0.021340614,-0.0066935504,-0.019698163,-0.0059566973,-0.036763903,0.008864735,0.030666586,0.03127407,0.0023666483,-0.036583908,0.018471949,0.050488494,0.011823397,-0.01122154,0.006209815,0.024884257,0.00022516907,0.0015862028,0.0072222846,0.0100797,-0.01645826,-0.03386149,0.004170813,0.006575429,-0.022094341,0.016199516,-0.011789649,0.037686378,0.03624642,-0.024186779,-0.0083753755,0.006963542,0.0076328972,-0.0142195765,-0.023196809,0.012318383,-0.01274587,0.0021866537,-0.022859318,0.00871849,0.043131214,0.03581893,-0.01576078,0.0017155738,-0.04394119,-0.013218356,0.0008254441,0.015873278,-0.054133385,-0.010625308,-0.018719442,0.027021695,-0.007644147,-0.029271627,-0.006350436,-0.0143770715,0.018066961,0.018044462,0.015243296,0.026054224,0.0201594,-0.03932883,0.000010519096,0.007008541,-0.024096781,0.04063379,0.03388399,-0.0010173916,0.0067497985,0.02920413,0.0017563539,0.0028771018,0.047833573,-0.0077510187,-0.04398619,0.040048804,0.03084658,-0.023028063,0.024276776,0.019315675,-0.0031442812,0.039306328,-0.019068182,-0.012520877,0.005889199,-0.03381649,-0.010726555,-0.0137470905,0.0073235314,-0.02582923,0.008988482,0.0008395062,0.023624295,-0.03719139,-0.0055854586,0.012678372,0.0058160764,0.0038080115,-0.05134347,0.00805476,-0.006440433,0.048013568,-0.0012951177,-0.006603553,0.028889138,-0.02384929,-0.023084313,0.004300184,0.0045589264,0.025311746,0.026751703,0.05237844,-0.010119073,0.00821788,0.0044014314,0.01745948,0.025244247,-0.01641326,-0.0054785865,0.016278265,-0.0060523194,0.013533346,-0.009629712,-0.022015594,0.029991606,0.0013626156,0.00528453,-0.017346982,-0.004162376,0.033389006,0.046708606,0.014534567,0.014849558,-0.0218356,-0.05161346,-0.01808946,-0.007644147,0.0029474122,-0.0005048287,-0.037866373,0.04223124,-0.021295615,0.034086484,0.020238146,0.008144757,0.0095790895,-0.0068285465,0.013792089,0.00854412,-0.020654384,-0.011542156,-0.027404184,-0.0035998926,-0.0028138224,-0.012363382,-0.0019419733,-0.011868396,0.0013133984,0.021869348,0.015884526,0.008589119,0.0045111156,-0.008600368,-0.008549745,-0.003838948,-0.037438884,0.049498525,0.009140353,-0.018888187,0.010619683,-0.0005090473,0.010709681,0.0036955148,-0.0059960713,0.01705449,0.024209278,0.020373143,0.033389006,0.022341834,0.0049161036,-0.039486323,-0.0073741553,0.03122907,-0.025649235,-0.022465581,0.010484687,-0.014107079,-0.03633642,0.0038164486,-0.030171601,-0.013454599,-0.021419361,-0.01749323,-0.0010033294,0.022015594,0.00872974,-0.049453527,-0.0120708905,0.010900925,0.0091347275,-0.014242075,-0.019180678,0.009854706,-0.020193148,0.03224154,-0.021565607,-0.015333293,-0.00019001386,0.009427219,-0.028304156,0.027899168,0.011530906,-0.013533346,0.0115196565,0.012475878,-0.014343322,-0.024681764,0.0036955148,-0.006181691,0.01240838,-0.004350808,0.039306328,0.035503943,-0.048733547,-0.032714024,0.022409331,0.023511799,-0.02147561,0.004075191,0.01642451,0.014827058,-0.0016621379,-0.020496888,-0.010962798,-0.016795749,0.0040245675,-0.023241807,-0.046078626,0.005318279,-0.005731704,-0.0005807639,0.042861223,-0.046483614,-0.01574953,0.0026872635,-0.07321282,0.018854437,-0.011733401,0.012678372,-0.027134191,-0.028574148,0.01879819,-0.0077678934,0.024411771,0.03892384,-0.0048289187,-0.038406353,0.010270944,-0.00076216477,0.023376804,0.010878426,0.005939823,0.0000028893182,0.040296298,-0.0029614742,0.025581738,0.011677152,0.015659533,-0.03055409,0.006575429,-0.0010595778,0.045763634,0.02040689,-0.00163542,0.019529417,0.039486323,0.008679116,0.013184607,0.0073629054,-0.029676614,0.008639743,-0.015007053,0.0049779764,0.022656824,0.0002733317,-0.07910764,-0.026684204,-0.015119549,-0.008926609,0.010293443,-0.0061535668,-0.017290734,0.006294187,-0.008594744,-0.021633105,-0.02452427,0.0018533822,0.055933334,-0.026571708,0.024704263,-0.018831939,0.012937115,-0.008797238,0.036988895,-0.0060523194,-0.0088197375,0.00067603454,-0.0076666465,0.030036604,0.01946192,0.028934138,-0.04958852,0.0065191807,-0.008409124,0.0068566706,0.013364602,0.021824349,0.026144221,-0.0014849558,0.004812044,-0.024884257,0.03125157,-0.0184607,-0.049768515,-0.013105859,0.021621855,-0.0010982485,-0.03287152,-0.005124222,0.011249664,-0.030396594,0.008909734,-0.017920716,0.01141841,0.011002172,-0.01776322,-0.006941043,0.024389273,0.007239159,0.0007818517,0.017178237,0.028776642,-0.007925388,-0.005939823,-0.030306596,0.006305437,0.00031903345,-0.01976566,0.0028292907,-0.028596647,-0.012644623,-0.000331162,-0.007340406,-0.025896728,0.008532871,0.0028264783,-0.02544674,-0.043221213,-0.01711074,0.022566827,-0.0014399571,0.0070985383,-0.002553674,-0.018595695,-0.008105383,-0.022049343,0.02148686,0.005419526,-0.0052957796,-0.00528453,0.025356743,-0.016390761,-0.010062825,0.009477843,0.07339281,0.04124127,0.013949584,-0.0059791966,-0.002858821,0.023354303,0.0054364004,-0.014275825,-0.026324214,0.02483926,0.004145501,-0.009050355,-0.0067160497,0.006603553,0.04495366,-0.0368314,0.049543522,-0.02816916,0.00770602,-0.0015749531,0.011424035,0.033636495,-0.0073010325,0.012363382,-0.016480759,-0.03253403,0.015063301,-0.013960834,-0.010642183,0.0402738,0.01473706,-0.0075147757,-0.03055409,0.008954733,-0.026369214,0.035323948,-0.02416428,0.010479063,-0.006462932,0.02652671,-0.005427963,-0.05296342,0.005076411,0.003574581,-0.023466801,0.0015355792,0.009005357,-0.05908324,0.016863247,0.011092169,0.033456504,-0.009865955,0.03482896,0.014838307,0.018066961,0.027764171,0.014287074,-0.005346403,-0.014557066,0.00974221,-0.018550698,0.006108568,-0.025604237,-0.01813446,-0.017819468,-0.0092191,0.00033872036,-0.0065360554,0.0028911638,-0.011857146,0.029766612,-0.0011474658,-0.0038895716,0.02447927,-0.013510847,0.014320823,0.010495937,-0.012183387,-0.00034030236,0.0040976903,-0.018246956,0.0369664,0.008628493,-0.0005628348,0.0013724591,-0.012487127,-0.036156423,0.015164548,0.0100572,0.05233344,0.027314186,-0.007537275,-0.034693964,0.019338174,-0.0035914555,-0.009528466,-0.031116573,-0.012610874,0.029316626,0.009674711,0.00028387827,0.00019106852,-0.0056051454,0.025964227,0.0061198175,0.0039120708,0.037483882,0.019315675,0.023005564,-0.0032792771,0.013004612,-0.015547036,-0.015367042,0.002796948,-0.030171601,0.025221748,0.02044064,-0.0059566973,0.029361624,0.014984554,0.05534835,0.0011341068,0.004277685,-0.013263355,0.013522097,-0.003810824,0.0016579194,-0.052918423,0.022578077,-0.0038670723,0.0023033689,-0.021025622,0.042793725,-0.0021121246,0.024051784,-0.03757388,-0.0035858306,-0.023376804,-0.0057401415,-0.008352876,0.014320823,-0.01274587,-0.03390649,0.06637302,-0.034153983,-0.0013450381,0.014714561,-0.0056670187,0.0035380195,-0.015580785,0.03660641,0.044931162,0.019191928,0.02787667,0.0007291189,0.031611558,-0.009607214,0.037416384,0.005079224,0.0064291833,-0.020631885,0.006963542,0.02044064,0.023376804,-0.013049611,-0.030576589,0.022105591,-0.023916787,0.021711852,0.0468886,0.006249189,0.006766673,-0.017346982,-0.0070310403,-0.017470729,-0.01576078,-0.049408525,-0.015007053,0.035728935,0.0071604117,0.026616706,0.0042945594,0.027606677,0.016998243,0.04967852,0.0035183325,0.01577203,-0.016345764,0.01644701,-0.039733816,-0.016357012,0.00078044547],
        "Category": "Boutique",
        "Tags": [
            "concierge",
            "view",
            "air conditioning"
        ],
        "ParkingIncluded": "true",
        "LastRenovationDate": "2020-02-06T00:00:00Z",
        "Rating": 4.60,
        "Address": {
            "StreetAddress": "7400 San Pedro Ave",
            "City": "San Antonio",
            "StateProvince": "TX",
            "PostalCode": "78216",
            "Country": "USA"
        },
        "Location": {
            "type": "Point",
            "coordinates": [
                -98.498923,
                29.518029
            ]
        }
    },
    {
        "@search.action": "mergeOrUpload",
        "HotelId": "48",
        "HotelName": "Nordick's Valley Motel",
        "Description": "Only 90 miles (about 2 hours) from the nation's capital and nearby most everything the historic valley has to offer. Hiking? Wine Tasting? Exploring the caverns? It's all nearby and we have specially priced packages to help make our B&B your home base for fun while visiting the valley.",
        "DescriptionVector": [-0.06868838,-0.01605626,0.03267631,0.005335447,-0.03286424,-0.012896689,0.04641868,0.04179091,-0.011739746,0.010717876,-0.014094742,0.017618427,-0.043952104,0.013507461,0.038666576,0.03366294,0.007687507,0.040592857,0.04291849,0.023244578,-0.026427642,-0.01954471,-0.04961349,0.0071707,0.05722465,-0.01691369,-0.011446105,0.015292794,0.002882081,-0.04900272,-0.030256713,-0.036199994,-0.025370535,-0.058822054,-0.008997143,0.04468033,-0.0004114637,-0.014646786,0.011610543,0.009643152,-0.0017794612,0.025605448,0.0058405087,-0.025957815,-0.0032182995,-0.010265671,0.012767487,0.04561998,-0.026474623,-0.0195682,-0.03422673,-0.05398286,-0.047146913,-0.05966774,-0.03704568,0.026944447,-0.030890975,-0.02050785,0.024430886,0.013671899,0.029035168,-0.022962684,0.017418751,0.05619104,0.005655515,0.047945615,-0.0013507461,0.018792989,-0.0045925365,-0.00863303,0.013190329,0.03953575,-0.018699024,0.0048949863,0.035424784,0.030632572,-0.0168902,-0.040029068,-0.008950161,-0.038267225,-0.0026530414,-0.019098375,0.03345152,-0.0034678937,-0.017219076,-0.0399351,0.009443477,0.036223486,-0.019603437,-0.0021655983,0.026357166,0.0006882198,0.020601815,-0.05356002,-0.030444643,0.045925368,0.03305217,-0.0019248131,-0.034320697,0.05703672,0.027014922,-0.058963004,0.002802798,0.0004624837,0.046230752,0.023044903,0.048157036,0.06469487,0.015163593,0.008327643,-0.078225814,-0.009519824,-0.002112743,0.03042115,-0.022116998,0.04028747,0.0050829165,-0.03267631,0.02621622,0.01841713,-0.01273225,-0.010988026,-0.042025823,-0.032981697,-0.007769726,-0.029434519,-0.05153977,0.02452485,-0.03596508,-0.02605178,0.008680012,-0.03037417,0.09894509,0.024407394,-0.0043135784,-0.017453989,-0.0019982234,-0.0076757614,-0.030045291,0.014764242,-0.00015939171,-0.011493088,0.013084618,-0.009308402,0.038666576,0.053841915,0.004842131,0.011363885,0.0076816343,0.025887342,-0.00080897944,0.020190718,-0.009960284,-0.031031923,-0.055768196,-0.026098764,-0.021623682,0.023949316,-0.012039259,0.05050616,-0.032253467,0.007147209,0.021964306,-0.036505383,0.023902332,0.012238934,-0.011093737,0.014364891,-0.042260733,-0.018511094,0.012556066,-0.035354313,0.011328649,0.011880693,0.0025532038,0.013436987,0.011011517,0.03396833,-0.0054910765,-0.0065716733,0.013566189,-0.015175339,0.023174105,-0.014987409,-0.014494093,0.00067757536,0.003629396,0.039394803,-0.026920957,0.016878454,-0.044962227,-0.021130366,-0.024924202,0.0000987366,0.0007553901,-0.03453212,-0.009901556,-0.010988026,0.0175597,0.028706292,-0.054781564,-0.03248838,0.07333964,-0.04545554,-0.043364823,-0.027531728,0.018099997,0.010805969,-0.00055718276,-0.019450745,-0.0034355933,0.029575467,0.025605448,0.036129523,0.012180206,-0.030115765,-0.044750806,0.042707067,0.021012912,0.007963529,-0.04616028,0.01538676,0.022739517,0.053043213,0.047828157,0.034320697,-0.03934782,-0.018569821,0.015809601,-0.0087269945,0.021435753,-0.03495496,-0.022669043,-0.008151459,-0.032605834,0.024642307,0.018992664,-0.0010541693,0.01235639,-0.010083613,-0.055016477,-0.029011676,0.0305621,0.033733416,0.0067830947,-0.01672576,0.005720116,-0.021752885,0.053137176,0.018511094,-0.048039578,-0.015398505,-0.01908663,-0.0436937,-0.014999154,0.023385527,-0.02093069,0.0044457163,0.026920957,-0.01624419,0.026451131,0.033991817,-0.02562894,-0.018605059,-0.010835333,-0.021987798,0.025159115,-0.0039670826,-0.025417518,-0.054640617,-0.034649573,-0.029998308,0.006178195,-0.010571056,-0.0006595899,-0.044774298,-0.0032036174,-0.020472612,0.04312991,-0.007746235,-0.010300907,0.0023359098,0.0032388542,0.015398505,-0.030233221,-0.021905579,0.04103919,0.038431663,-0.019438999,-0.006548182,-0.04564347,0.03702219,-0.009831082,-0.009883937,0.028588835,-0.01986184,0.028729782,-0.031501748,0.020519596,0.021153858,-0.0059051095,0.04157949,0.017994286,0.0043429425,0.03171317,0.014000777,-0.021071639,-0.008186696,0.0010460941,0.021999544,0.019215832,0.023068395,-0.0129084345,0.005746544,0.018746007,-0.038807523,-0.06173497,-0.020590069,0.063520305,-0.025417518,0.036975205,0.02562894,0.008433354,-0.0055997237,-0.046089806,-0.013166838,0.052385457,-0.021494482,-0.0025047532,-0.009284911,0.027085396,-0.021529717,-0.02833043,0.05600311,-0.01900441,-0.015046136,-0.038995452,-0.011440232,0.03840817,0.059385847,-0.05168072,-0.010148214,-0.036223486,0.009478714,-0.026192728,0.039629716,0.008398117,0.029223097,0.0071237176,-0.0039230366,-0.07084957,0.001154741,0.0012428332,0.020672288,-0.0023917016,-0.007998766,0.053700965,-0.044586368,-0.033334065,0.0024504296,-0.029411027,-0.012180206,0.02983387,-0.017512716,0.012720505,0.03706917,0.019239323,0.004977206,-0.0034091657,-0.01281447,0.03020973,-0.0032417907,0.01262654,-0.0025091576,0.018581567,0.053653985,0.06577546,0.011804346,-0.0054176664,-0.01758319,-0.016385138,-0.031478256,0.018276181,0.0033298829,0.047640227,-0.05698974,0.011857201,-0.046653595,-0.061828934,0.006430726,0.039700188,-0.0132373115,-0.06333237,-0.008685885,-0.024242956,-0.032605834,0.029199608,-0.015363269,-0.0053824293,0.045126665,0.027531728,-0.03514289,-0.058070336,-0.014905189,0.012591302,0.0170194,-0.012509083,-0.03363945,-0.02605178,0.02508864,-0.014881698,-0.055768196,0.052291494,-0.040733803,0.01034789,0.03685775,-0.015797857,-0.037585977,0.03587112,-0.022175727,0.0123328995,0.059714723,0.015504216,-0.00021142112,-0.06572848,0.0016796234,0.02945801,-0.013319531,-0.08395768,-0.026944447,-0.0009792909,-0.02230493,0.036740292,0.062862545,0.00403462,0.010048376,-0.06474185,-0.014447111,0.014752496,-0.007664016,0.0070003886,-0.029129133,-0.05191563,0.012485592,0.031854115,0.009696008,0.03897196,-0.0019321542,-0.01747748,-0.030820502,0.019204086,-0.027813625,-0.03345152,-0.04620726,-0.00024555682,0.03932433,0.00018701227,-0.01793556,0.019626928,-0.018053016,0.01726606,0.008797468,-0.046653595,0.013472224,-0.00175597,-0.039817646,0.0058170175,0.011810219,0.025605448,-0.049378578,0.005203309,-0.028212976,-0.01586833,-0.008386372,-0.039230365,-0.015997533,-0.05891602,0.040216997,-0.033733416,-0.01720733,0.002265436,-0.004586664,-0.0042284224,0.039606225,-0.00702388,-0.104395054,-0.02385535,-0.010835333,-0.06704399,-0.028424395,0.039206874,0.014916935,-0.019626928,-0.01064153,0.010300907,0.027296817,-0.05088202,0.044327963,0.043482278,-0.014458856,-0.0065775462,-0.038619593,0.017042892,0.00021637631,0.007687507,-0.0063308883,0.014517584,0.013260803,0.0008574301,0.021048147,-0.013272548,-0.015457233,-0.0043634973,-0.006048993,-0.0074643404,0.013354768,0.0074291034,0.027461255,0.004008192,0.046818033,0.021153858,0.013225566,0.010506456,0.0044310344,-0.019955805,0.03481401,0.009543315,-0.039512258,0.031595714,0.0008449504,-0.018652042,-0.019603437,-0.033005185,-0.00601082,-0.010130595,0.049331598,0.0071824454,-0.018769497,0.013906812,0.01804127,0.002895295,0.00038063145,-0.011628162,0.01538676,-0.00027749024,-0.027226344,0.00060526637,0.020601815,0.012708759,0.012931925,-0.003629396,0.051962614,-0.02945801,-0.024172483,0.017512716,0.0037292338,-0.0057582892,0.014541076,-0.022845227,0.011974658,0.010342017,0.0029716415,0.024830237,0.008450972,0.057271633,0.02184685,0.034297206,0.037867874,-0.014353146,-0.017042892,0.024618816,0.037092663,-0.0054763947,-0.00027969253,0.05417079,0.0008383435,0.032840747,-0.041438542,-0.017324787,0.00765227,0.006307397,-0.00034245817,-0.018945683,-0.008544937,-0.0019747321,-0.061687987,0.026944447,-0.041250613,-0.0024871347,-0.0019497726,-0.025347045,0.025981307,0.0021465118,0.0050858525,-0.013601426,-0.0066245287,-0.0010049845,-0.04009954,-0.039042436,-0.017148603,0.013577934,-0.01798254,0.0047011836,0.016326409,-0.021705903,-0.007164827,-0.0215767,0.018546332,0.00631327,0.0050418065,0.0031889353,-0.011181829,0.028541852,-0.01795905,-0.0026222093,0.02098942,-0.019826604,-0.009836955,-0.022763008,-0.016878454,0.04580791,-0.010676767,-0.0058757453,0.015844839,0.0023461871,0.02640415,-0.01793556,-0.04291849,-0.008186696,-0.009226183,-0.0026780008,0.05722465,0.049472544,-0.040216997,-0.028283449,-0.01871077,-0.009754736,-0.002808671,-0.018428875,-0.012943671,0.014411873,0.018781243,-0.0064718355,0.008544937,-0.0050917254,0.018804735,0.015927058,-0.003905418,-0.01833491,-0.03932433,0.019615183,0.028447887,0.01292018,0.01795905,0.012051004,0.027860606,-0.008169077,-0.020519596,-0.033287082,-0.0003586084,0.0010805968,0.017677156,-0.015363269,0.03114938,0.00601082,-0.0024093199,0.006800713,-0.0116810175,-0.02335029,0.044562876,0.05929188,0.024665799,0.020178972,0.0012443014,-0.000754656,0.009185073,0.04446891,0.00009937894,0.020860218,0.0059491554,0.050929,-0.0029877916,-0.0075583053,0.039841138,-0.020061515,-0.02098942,0.017806357,-0.0017089874,0.0015445488,-0.004636583,-0.011181829,-0.024289938,0.0037938347,0.011980531,0.033287082,-0.02887073,-0.023127122,-0.022563333,0.019450745,-0.091474876,0.00072272256,-0.028706292,0.02042563,-0.023608692,0.0017941432,-0.029223097,-0.023902332,-0.017066384,0.014576312,-0.017042892,-0.02393757,-0.019027902,-0.034673065,0.0037321702,-0.017089874,-0.034696557,-0.021459244,-0.004848004,0.018652042,-0.050036334,-0.025347045,0.027813625,-0.0012068623,0.0019262814,-0.00055057585,0.012156715,0.015985787,-0.04942556,-0.00038283374,0.07733315,0.025159115,-0.031501748,0.01139325,0.045103174,0.011316903,0.011669272,-0.003861372,-0.0082336785,-0.057459563,-0.025206096,0.015551198,0.015410251,0.003723361,-0.008245424,0.016467357,-0.00859192,0.02927008,-0.025558464,-0.07315171,0.021553209,-0.013601426,0.008022257,-0.013824592,-0.03666982,0.014317908,-0.026920957,0.004298896,0.013319531,-0.0027969254,-0.004122712,-0.007875437,-0.013178583,-0.000082448736,0.03516638,-0.0038525627,0.050083317,0.008409862,0.025981307,-0.018323164,-0.030632572,0.0065658004,0.035048924,-0.03363945,-0.029669432,0.012884943,-0.0032770275,0.016984165,-0.050224263,0.017007655,-0.019579945,0.046771053,-0.004798085,-0.0046835653,-0.00010837168,-0.0041843764,0.019016156,0.00075612415,-0.015797857,0.016114987,0.0018337846,-0.019180594,-0.0313608,0.02544101,-0.056378968,0.009696008,-0.045901876,-0.005582105,-0.05647293,0.028894221,0.025605448,0.042307716,-0.025041658,0.028259957,-0.029622449,0.039441787,-0.0056525785,-0.033287082,-0.02640415,-0.004921414,-0.037398048,-0.012191951,-0.0125795575,-0.0019468362,-0.030350678,0.032441396,0.021811614,-0.021705903,0.012426864,0.037867874,-0.011510706,-0.026897466,0.013507461,-0.03589461,0.0037145517,-0.019333288,-0.016526084,-0.0012949544,0.0021024656,-0.014576312,-0.05910395,-0.03154873,0.003077352,0.022234455,0.0038231986,-0.014364891,-0.0034062292,0.018111743,0.0209072,0.014846462,0.03683426,0.035283837,0.022023033,0.005869873,-0.023679167,0.0011143655,-0.032464888,0.023808368,0.0077403625,-0.006882932,-0.008803341,0.0039142272,0.010753114,-0.009143963,0.026075272,-0.022187473,-0.023925824,0.00034355934,-0.016279427,0.028823746,-0.011769109,-0.007628779,-0.034461644,0.0023872969,0.043035947,0.007922419,0.016173717,0.013472224,0.00015535415,-0.026756518,0.029434519,-0.07380947,-0.021917323,0.034508627,0.024853729,-0.009766482,-0.037679944,0.009613789,-0.041180138,0.016114987,-0.033615958,0.0134839695,0.017771121,-0.0013903875,0.02034341,0.0035970956,0.016279427,0.003033306,0.045878384,0.024642307,0.021071639,-0.000016804033,-0.00015691412,0.018428875,0.014975663,-0.017242568,0.01425918,0.012931925,0.040804278,-0.024289938,0.032206483,-0.00034245817,-0.006172322,-0.015057882,0.018276181,0.001198053,-0.018781243,-0.012532575,0.02109513,-0.010083613,-0.051304862,-0.010130595,-0.055909142,0.0001237878,0.029974818,0.009772355,0.009519824,-0.021635428,0.03627047,-0.021377025,-0.02385535,-0.01881648,0.018064762,-0.022363657,0.039982084,0.009108727,-0.005050616,-0.016044514,-0.020543085,0.020719271,-0.0029305317,-0.0012773359,-0.008820959,-0.005567423,-0.009725372,0.04052238,-0.0064542172,0.02736729,0.01191593,-0.0049390323,0.01262654,-0.025511483,-0.04031096,0.03589461,-0.05017728,-0.008944288,0.013519206,-0.006912296,-0.010712003,0.021705903,-0.017219076,0.019274559,-0.0003224539,0.0093025295,-0.00031052477,-0.025910834,0.047240876,0.020648796,0.0073586297,-0.014129979,-0.015645163,0.0037850256,-0.019709148,0.014376637,-0.009167455,0.00003854031,0.0013030295,0.038478646,0.008832705,0.021153858,0.023843605,0.010571056,0.009977902,0.002569354,-0.004980142,-0.00884445,0.005438221,0.039817646,0.023456,-0.023714403,-0.048204016,0.028823746,0.06056041,-0.007658143,0.03227696,-0.0064718355,0.018792989,-0.036340944,0.019204086,0.03495496,-0.013707137,0.008433354,-0.016326409,0.012191951,-0.014482347,0.013648408,0.009143963,0.026873974,0.047381822,-0.007294029,-0.0032946458,-0.008909051,-0.03441466,-0.0071119717,0.0075876694,0.0033063914,0.027108887,-0.009014762,0.032958206,0.0031155252,0.0041990583,0.0170194,-0.04294198,-0.017759375,0.015856585,-0.040545873,-0.00418144,-0.030890975,0.0044545257,-0.006788967,0.024266448,0.034461644,-0.0123328995,-0.013765864,0.016584814,0.007006261,-0.029763397,0.014705514,-0.009619662,-0.0009257015,0.002184685,-0.028400905,0.0006540841,-0.0000064119145,0.01954471,0.016984165,-0.001952709,-0.0066186558,-0.0114343595,-0.006031375,0.024172483,-0.014846462,-0.006066612,-0.0007447456,-0.014881698,-0.014188707,0.016561322,0.013671899,0.035072416,0.023150614,0.05074107,0.006583419,-0.018933937,-0.013131601,-0.0046630106,0.029904343,-0.020460866,-0.026920957,-0.009437604,-0.0032975823,-0.005552741,0.023174105,0.022856973,-0.01908663,-0.037891366,-0.04446891,0.01624419,0.026756518,0.010864696,0.010060122,0.038267225,0.01463504,-0.005147517,0.024102008,-0.0046571377,-0.023596946,-0.04031096,0.005623215,-0.0007454797,0.018076506,0.03800882,0.0069768974,-0.032417905,0.019873586,0.008462718,-0.02833043,-0.020977674,0.021929068,0.0106943855,-0.010946916,0.03232394,-0.00038246668,-0.0058170175,-0.029223097,0.017031146,0.003156635,0.013871575,0.0034796393,0.056238018,0.013895066,0.01938027,-0.0010071867,0.0076816343,0.038854506,0.0026970876,-0.0089736525,-0.0063308883,-0.034179747,0.029551975,0.0124151185,-0.006189941,0.032746784,-0.028447887,-0.0071237176,0.025534974,0.010295034,0.016502593,0.0015122483,-0.021083385,-0.020777998,0.015245812,-0.0042695324,-0.017818103,-0.026122255,0.0013551507,-0.011117227,-0.008785723,-0.017794611,-0.003970019,-0.023326797,0.03596508,0.01881648,-0.018652042,-0.021647174,-0.009989648,0.0035090034,-0.020120244,0.0064659626,0.010841206,0.020531341,0.012403373,0.012697013,0.006060739,0.0061312127,-0.017418751,0.015445488,-0.015234067,0.007928292,0.059996616,-0.01662005,-0.020860218,-0.0059609013,-0.009907429,0.032253467,-0.031266835,-0.019720893,-0.012027513,0.01131103,0.010606294,0.0011503365,0.0019996916,0.0064953268,0.0023770195,-0.02452485,-0.015856585,0.0031830624,-0.026498115,0.008621284,0.00873874,-0.034438152,-0.013425241,-0.010770732,-0.03455561,0.0038349442,0.0019130675,-0.01774763,0.017994286,-0.01766541,-0.0036528872,-0.018640297,-0.014024268,0.011052627,0.023620438,-0.0065658004,0.03436768,-0.017677156,0.024031535,0.0057318616,0.042589612,-0.0013845147,0.004601346,-0.019756129,-0.012767487,0.029387537,-0.009566806,-0.031619202,0.0058111446,0.029857362,-0.03152524,0.024736272,0.015187085,0.003946528,-0.019885331,-0.032934714,0.03230045,-0.057976373,0.027273325,0.0088092135,0.0033504376,-0.011868947,0.003925973,0.013319531,0.013096364,-0.05525139,-0.0029143814,0.014623295,-0.026756518,0.041156646,-0.031948082,-0.029340554,0.016361646,-0.033498503,0.0113521395,-0.007822582,-0.007945911,0.013918557,0.017970797,-0.05074107,-0.0030303695,0.017806357,-0.012685267,-0.00015113308,0.033357557,-0.016772743,0.035448276,0.014423619,0.008298279,-0.00877985,-0.0029877916,0.028776765,-0.025487991,0.005802335,-0.01385983,0.0049595875,-0.00030850602,0.027249834,0.020073261,-0.03363945,0.023021411,0.0035236855,0.004848004,0.024008043,-0.0137306275,-0.013319531,0.013178583,-0.026850482,-0.03342803,0.06253367,0.012708759,0.022257946,-0.009919175,-0.02385535,-0.011299285,0.00863303,0.04108617,0.010747241,-0.030350678,0.02320934,-0.030256713,-0.026380658,0.029857362,-0.052291494,0.006800713,-0.01262654,0.020038025,0.0055409954,-0.0031037796,-0.0046718195,0.02544101,0.017336532,-0.0014124106,0.040193506,-0.016655287,0.0017383515,-0.0115987975,0.033615958,0.033545487,0.001444711,-0.016232444,0.007476086,-0.011117227,-0.021893833,0.0013639599,0.0005604862,-0.044140033,0.034297206,0.013307786,-0.040945224,-0.013942049,0.0675608,-0.0014549885,-0.023690911,-0.0040463656,0.022046525,0.017125111,0.02621622,0.013636663,-0.0060049472,-0.00974299,0.0040170015,-0.0038202624,-0.008333516,0.0021817486,0.002009969,0.028048536,-0.0037116155,-0.014541076,-0.017160349,-0.0075876694,-0.012344644,0.003934782,0.015786111,0.010265671,0.017794611,-0.0012545788,-0.023784876,-0.012180206,-0.044703823,-0.018064762,0.006336761,-0.0059051095,0.014834716,0.01929805,-0.024430886,-0.02659208,-0.008903178,-0.014682023,-0.010600421,-0.01814698,-0.028964695,-0.020296428,0.03356898,-0.012368136,0.012967163,0.008257169,0.009977902,-0.004290087,0.0007781472,-0.018734261,-0.03232394,0.0042196135,-0.018076506,-0.0036910605,0.026615571,-0.019168848,0.026169237,-0.022070017,-0.0014417747,-0.018182216,-0.010835333,0.0006588558,-0.04087475,0.015128356,-0.007564178,-0.015750874,-0.01154007,0.03020973,0.022210963,-0.033169627,-0.035800643,-0.018311419,-0.011792601,-0.00048744315,-0.011545943,-0.019697402,0.017430497,-0.031948082,0.0047011836,-0.023115376,0.046865016,0.012051004,-0.027132379,0.007258792,-0.009096981,-0.0041961223,-0.01358968,0.038337696,0.0055703595,0.011551815,-0.0155277075,-0.002061356,-0.013096364,-0.019063137,0.0064953268,0.00054543716,-0.0041050934,-0.007752108,-0.011639908,-0.03683426,-0.022833481,0.004833322,-0.014024268,-0.010259798,-0.0034972578,-0.010623911,-0.015163593,-0.008086858,-0.0027749024,-0.032182995,-0.0281425,-0.037280593,0.0134017505,0.009378877,0.0058405087,-0.016279427,0.0115987975,-0.013566189,0.029387537,-0.003150762,0.017794611,-0.0028629943,-0.036223486,0.0056525785,-0.016737506,-0.0064953268,0.021330042,0.03114938,-0.021083385,-0.021236077,0.026169237,0.015328032,-0.06690304,-0.014764242,0.014129979,0.024665799,0.0017868022,-0.022586824,-0.011692763,-0.028847238,0.0038554992,0.00320949,0.007998766,-0.03702219,0.06347332,0.000086211,0.032394417,0.007570051,0.020672288,0.013260803,0.040545873,-0.029411027,0.0004745964,0.018381892,0.028025044,-0.018734261,0.041555997,-0.018558078,-0.026639063,-0.000029960502,0.017547954,0.022539841,0.032723293,0.014423619,-0.035401292,0.024877219,0.05492251,0.0014674682,0.023385527,-0.021517973,0.0019218768,0.03439117,-0.012344644,-0.06023153,-0.030092273,0.002497412,0.03098494,0.0069357874,-0.008644775,0.004157949,0.008380499,0.012556066,-0.019932315,-0.009919175,0.008562556,-0.008644775,0.019016156,0.019027902,-0.0020892518,0.048251,-0.027085396,-0.0049243504,-0.018640297,0.034673065,-0.00044266297,-0.000776679,0.019603437,0.052385457,0.007282283,0.0024636434,0.0013558848,-0.002149448,-0.012591302,0.05567423,0.032629326,0.0458314,-0.047146913,-0.007387994,-0.00061003806,-0.0052649733,0.004125648,-0.01758319,-0.014588058,-0.021635428,0.03462608,-0.0037174881,-0.010946916,0.012121478,0.020178972,0.017806357,-0.016009277,-0.0064953268,-0.035824135,0.0057083704,-0.035095908,-0.009789973,0.02088371,-0.048297983,-0.005482267,0.007722744,0.02774315,0.007176573,0.0062897787,0.0120862415,0.018182216,-0.04200233,-0.00013342289,0.0023241641],
        "Category": "Boutique",
        "Tags": [
            "continental breakfast",
            "air conditioning",
            "free wifi"
        ],
        "ParkingIncluded": "false",
        "LastRenovationDate": "2018-02-19T00:00:00Z",
        "Rating": 4.5,
        "Address": {
            "StreetAddress": "1401 I St NW",
            "City": "Washington D.C.",
            "StateProvince": "null",
            "PostalCode": "20005",
            "Country": "USA"
        },
        "Location": {
            "type": "Point",
            "coordinates": [
            -77.03241,
            38.90166
            ],
            "crs": {
            "type": "name",
            "properties": {
                "name": "EPSG:4326"
                }
            }
        }
    },
    {
        "@search.action": "mergeOrUpload",
        "HotelId": "49",
        "HotelName": "Swirling Currents Hotel",
        "Description": "Spacious rooms, glamorous suites and residences, rooftop pool, walking access to shopping, dining, entertainment and the city center. Each room comes equipped with a microwave, a coffee maker and a minifridge. In-room entertainment includes complimentary W-Fi and flat-screen TVs.",
        "DescriptionVector": [-0.042438243,-0.016445654,0.031978894,0.014899005,-0.034515843,-0.018871332,-0.0060530687,0.004514766,0.021452786,-0.004854138,-0.00635906,0.01942768,0.010164482,-0.024857638,-0.02872982,0.03224594,-0.08630073,0.040235102,0.04464138,0.059195448,-0.004511984,0.007293725,-0.050516415,-0.025814557,-0.03353667,0.04793496,-0.058171768,0.013619404,0.002342226,0.026081603,-0.0065204008,-0.02081855,0.055545803,0.0077165496,0.027906425,-0.029642232,0.013552642,-0.028173473,0.029976042,0.010615123,0.007377177,0.038388025,-0.029152647,0.002190621,0.009535808,-0.031244515,-0.010186736,0.022331817,0.01183909,0.051139526,-0.021920118,0.032824542,-0.03467162,-0.06738489,-0.015433099,0.008617834,-0.024011988,-0.06168789,0.0066094166,-0.012028248,-0.016412271,-0.004717833,-0.00071351655,0.065782614,-0.020150932,-0.0036718983,-0.013730674,0.05416606,-0.016935239,0.007199146,0.01865992,-0.008901571,0.00586391,-0.019283028,0.014309276,-0.004884737,-0.02939744,-0.012161772,-0.015088163,-0.017725253,-0.033959493,-0.026014842,0.013541515,0.064892456,0.018615412,-0.0016440089,-0.029019123,-0.024857638,-0.010370331,0.08242855,0.00936334,0.014008848,-0.02939744,-0.0045481464,-0.060797732,-0.013942086,0.050516415,-0.009285452,-0.034938667,0.102635115,0.054077044,-0.07601942,-0.01334123,-0.007199146,0.041548084,-0.04043539,0.044218555,-0.016768334,0.054121554,0.008050359,-0.100409724,-0.01759173,-0.039233677,-0.022854785,-0.01169444,0.006971043,0.026704714,-0.0396565,-0.021797722,0.011388448,0.026326397,0.03745336,0.0032212562,-0.044730395,0.00016334036,-0.069877334,-0.015344083,-0.015900431,-0.03834352,0.02821798,-0.032156926,-0.006820829,0.003908346,-0.0040696873,0.011271615,0.067028835,0.0042115557,0.0042477185,-0.002353353,-0.051451083,0.006670615,0.0075273914,0.021530675,0.015666766,-0.00064223446,-0.01865992,0.013096437,0.027505856,-0.05812726,0.042838812,-0.013051929,0.022109278,0.0043200436,0.0035578469,-0.004717833,-0.027327824,-0.01198374,0.015666766,-0.015533242,0.033803716,0.000098925666,-0.026370905,-0.04090272,-0.016734954,0.0051184036,0.012974041,-0.018637665,0.040079325,-0.052474763,-0.028574044,0.05109502,-0.00897946,-0.004648289,0.040546656,0.03823225,0.031177754,0.021964626,0.012751501,0.008028105,-0.015855923,0.00059146766,-0.025347224,0.050783463,-0.018938093,-0.013908705,0.019071616,-0.056569487,0.013196579,0.0064425124,0.030287595,-0.025747795,0.00039744124,-0.032958068,0.0027094157,-0.045932107,0.059507005,-0.0645809,-0.007026678,0.0031990022,-0.008812556,-0.037742663,-0.03745336,0.04855807,-0.01660143,-0.02599259,-0.04882512,-0.024590591,0.041147515,0.024568336,0.0021113413,0.018715553,-0.017970048,0.01824822,0.04533125,-0.017257921,-0.008634524,-0.02169758,-0.0016273186,0.0062978617,0.052830826,0.00030338363,0.024902146,0.005541228,0.025681034,-0.034248795,0.031311277,-0.0039361636,-0.030220835,0.014943513,-0.0041002864,-0.008957206,-0.018926965,-0.03700828,-0.022632245,-0.039055645,0.01920514,-0.029864771,0.015377465,0.0074161217,0.0016328819,-0.029820263,-0.016746081,0.008985024,0.038388025,0.04753439,0.0014798862,0.031756356,-0.03516121,0.029820263,-0.032713275,-0.03484965,-0.0040029255,0.014887878,-0.01620086,0.02209815,0.013330103,0.04468589,0.013241087,0.02763938,-0.03262426,0.07054495,0.023655925,0.0039806715,-0.027327824,0.01418688,0.0022490376,-0.07695408,0.019383172,-0.004595436,-0.034538098,-0.065871626,-0.026726969,-0.0013651394,0.015766907,0.036229394,0.0050738957,0.01689073,-0.032490734,-0.019861631,0.033625685,0.00045307606,-0.033403147,-0.00094787823,-0.015010275,0.018014556,0.023767196,0.036073618,0.015566623,-0.051228542,-0.0020807423,0.022120405,-0.025903573,0.0077221133,-0.06426934,0.0055467915,0.02741684,0.028017696,-0.012662485,0.031467054,0.003716406,0.0029152646,0.028707568,-0.0065815994,-0.008634524,-0.044218555,0.019872759,-0.020640519,0.0061810287,0.01993952,-0.03213467,-0.010820973,0.004890301,0.026860492,-0.071034536,-0.016868478,-0.012862771,0.0066094166,0.03573981,-0.010876607,-0.03625165,0.0018429034,0.010047649,0.0038332392,-0.018737808,-0.015533242,-0.043595444,0.017002001,0.03206791,-0.05416606,-0.0645809,0.08033668,-0.007087876,0.021864485,-0.016668193,-0.027461348,-0.0005031474,-0.046599727,0.001794223,-0.009652642,0.0019694727,-0.012072756,-0.03954523,-0.026526682,0.02158631,0.02906363,-0.050827973,-0.022921545,0.016334383,0.05461114,0.017947793,-0.0060864496,-0.015188306,-0.020184312,0.02022882,0.007466193,-0.0018442943,0.05376549,0.010609561,-0.063690744,-0.007922399,-0.015644511,0.03602911,0.039122406,0.0029903715,0.038321264,0.03324737,-0.07152413,0.048157502,-0.034248795,-0.013096437,0.028685313,0.03602911,0.010726393,0.008005851,-0.0317341,0.03489416,0.026771476,0.03478289,-0.0135081345,-0.019750362,0.014164626,0.0030654785,0.0015758564,-0.02345564,0.005869474,0.0041308855,-0.018192586,0.0026009278,-0.024368051,0.04039088,-0.015121545,-0.014609704,-0.020774042,0.011916978,0.002382561,0.024590591,0.02254323,0.004690015,-0.0038582748,-0.0017205068,-0.0028846655,-0.08095979,0.021630818,-0.028418267,-0.011154781,-0.08403084,-0.0057415133,-0.07459517,0.0026440448,-0.007377177,-0.026704714,-0.07330444,0.0011891943,0.0043756785,-0.042505004,-0.024345798,-0.003040443,-0.012762628,0.045108713,-0.011288305,0.047890455,0.028418267,-0.020351218,0.024078751,-0.029775755,0.014164626,0.027861917,-0.054922696,-0.08082627,0.0065037105,-0.02532497,-0.0054689026,0.018481888,0.053008858,-0.017213413,0.012873897,-0.019783743,-0.020874185,-0.018493015,-0.032334957,-0.0039973618,-0.017246794,-0.06831956,0.0023783885,-0.0055885175,-0.004776249,0.007466193,0.06769645,0.014554069,-0.03941171,0.004606563,-0.028106712,-0.015510988,-0.01663481,-0.015533242,-0.00910742,0.023188593,-0.0034187597,-0.023477895,0.009975323,-0.0012643012,0.00872354,-0.016334383,-0.007226963,0.014832243,-0.050516415,-0.051228542,0.025191447,0.0084620565,-0.0012656922,-0.060263637,-0.05225222,0.025725542,0.031222261,0.023010561,-0.06716236,0.011916978,0.03375921,0.03008731,0.008818119,0.0034716127,0.013741801,-0.05421057,-0.0051712566,-0.008801429,-0.020317836,-0.05171813,-0.021608565,-0.05087248,0.0007510701,-0.023366624,0.016223114,-0.0037831678,-0.008139375,0.035495017,0.015922686,-0.059685037,0.0050405147,0.015900431,0.00504886,-0.023144085,0.0021447223,-0.0072992886,0.012384311,-0.0040724687,0.04508646,-0.030643659,-0.024546083,0.022131532,0.007939089,0.010904425,0.003952854,-0.026905,0.02158631,0.021742089,0.0317341,-0.0052574906,-0.015166052,-0.0018707209,-0.0373866,0.0033881606,-0.0075997165,0.019349791,-0.0027163702,0.015332957,0.027950933,0.0017664055,-0.053053364,0.014887878,-0.0023060634,-0.018481888,-0.010954496,-0.0036440808,0.008940516,0.020673899,0.016690446,-0.008934952,-0.015766907,0.06720687,-0.022309562,0.062355507,-0.0055996445,0.008406421,-0.047667913,-0.02045136,-0.023833957,-0.018737808,0.0004965408,0.0056441524,0.020918693,0.015922686,0.01891584,0.028685313,0.004831884,-0.009864054,0.0054522124,0.015154925,0.0043784603,0.0063924408,0.01653467,-0.019227395,0.033981748,-0.0064369487,0.021908993,0.024056496,0.019527823,0.030398866,0.039745517,0.0016314911,-0.0076776054,-0.0018915839,0.004879174,-0.008823683,-0.011266051,0.023121832,-0.015855923,0.008200573,-0.011627678,-0.021541802,0.0066038533,0.013608277,-0.010943369,0.070856504,0.028351504,-0.011672186,-0.038388025,0.022198293,0.00599187,0.034827396,0.017836524,0.05047191,0.004709488,0.029953787,-0.03756463,0.011154781,-0.010075466,-0.03874409,0.005824966,-0.031088738,0.026482174,-0.018203713,0.0025703288,-0.030843945,-0.008884881,0.0064202584,-0.009023968,-0.01235093,-0.013296722,-0.007338233,-0.038098726,-0.012484454,-0.008779175,-0.013330103,-0.00038735743,-0.03456035,0.04339516,-0.033692446,-0.034649365,0.010409275,-0.0006161556,-0.007215836,0.0144873075,0.018626537,0.038833104,0.020840803,0.039589737,-0.013719547,-0.044396587,0.0013018548,-0.04159259,0.048157502,0.030510135,0.058260784,0.008050359,0.0032880178,0.058750372,-0.014398292,0.017881032,-0.0077833114,-0.019772615,0.009908562,-0.0038304573,-0.0004770686,0.004587091,0.014453926,0.018637665,0.046866775,-0.023811704,0.0013498398,0.016746081,0.0033492162,-0.016901858,0.016846223,-0.03705279,0.0024785313,0.026504429,0.029820263,-0.0030265343,0.023767196,0.036563203,-0.023411132,0.025814557,-0.015889306,-0.002148895,0.0007246435,0.012484454,-0.016390018,0.02129701,-0.021397153,0.03903339,0.06876464,0.0033325257,0.0020626609,0.018782316,0.035450507,0.010047649,0.04226021,-0.02019544,-0.0022490376,0.026037097,0.008183883,0.017291302,-0.056124408,0.008834809,0.0015313484,-0.04159259,0.034449082,0.0032435101,0.039055645,-0.0052686175,-0.0018081317,-0.0018289947,-0.0042699724,0.016245367,0.0026273543,-0.041214276,0.00060607184,0.0037108425,0.0186933,-0.03698603,0.013174325,0.024657352,0.048469055,-0.010409275,0.0027247153,-0.014298148,0.02815122,-0.037653647,0.029508708,-0.018748935,-0.005930672,-0.013908705,0.001450678,-0.024746368,0.013797436,0.004965408,-0.01931641,-0.0076497877,0.0054967203,-0.009157492,0.004445222,0.032980323,-0.0018526395,0.02209815,-0.016935239,-0.0041948655,-0.023722688,-0.01469872,-0.01850414,0.04350643,0.012885025,0.006715123,-0.008473183,-0.0013727891,0.019917266,0.0006780494,-0.00078584184,-0.014921259,-0.021864485,-0.011894724,0.0009228426,-0.02396748,-0.018737808,-0.02345564,0.030154074,-0.034916412,0.018871332,-0.043773476,-0.057637673,0.020473614,-0.047000296,0.0059139812,0.022009134,-0.031110993,-0.028774329,-0.024167767,0.033158354,0.0132744685,-0.02059601,-0.0031461492,0.02056263,0.0045592734,0.009096293,0.004996007,0.032312702,-0.03516121,0.0063089887,0.013997721,-0.01784765,-0.0050961496,-0.008005851,0.030198582,0.029998295,-0.011210416,-0.024501575,0.0053159073,-0.014342656,-0.053453937,0.026593445,-0.0016064554,0.04317262,-0.009886308,0.024523828,-0.010292442,0.019739235,0.00011300823,-0.0036496443,-0.027350077,0.005824966,-0.011738948,-0.0042950083,-0.004973753,0.005187947,-0.02792868,-0.010275751,0.029041376,-0.00084773556,0.015099291,0.04457462,0.007076749,-0.0031350222,0.011099147,0.04159259,0.012506708,0.027995441,0.00896277,-0.0075496454,0.01920514,0.014453926,-0.042638525,-0.021029962,0.033380892,0.0020668337,0.003502212,-0.007922399,0.0066261073,0.0048597017,0.0012197935,0.0029820264,-0.009079603,-0.0030460064,-0.027973188,0.00013734847,0.03910015,0.0051962924,0.015332957,-0.020484742,-0.012373184,0.014821116,-0.029731248,0.015277321,0.010164482,-0.016946366,0.032958068,0.00080531405,-0.009674896,0.013152071,0.04628817,0.004050215,0.007087876,0.005797148,0.014809989,-0.011227107,-0.00030338363,-0.012250788,-0.023188593,-0.019160632,0.014287022,0.006325679,0.0013797436,0.048335534,-0.0014882315,0.02679373,-0.022365198,0.020940946,0.018370617,0.019984027,0.020150932,0.008617834,0.0065927263,-0.0373866,0.0039695445,-0.004756777,0.0002465318,-0.03496092,0.03262426,-0.0011224325,0.00044264455,0.012829389,0.032223687,-0.0025911918,-0.013686166,-0.007916835,0.028084457,0.032980323,-0.008812556,0.008806992,0.009391158,0.021174613,-0.05643596,0.014921259,0.012517835,-0.0013560988,0.016134098,-0.018737808,0.021675326,-0.021285882,0.010509417,-0.003911128,0.0016301002,-0.0031600578,0.013007421,0.027238809,-0.019728107,-0.02815122,0.019616839,-0.011154781,0.0271943,-0.01195036,0.014720974,-0.028596297,0.0043005715,-0.020863058,-0.032713275,-0.00005333119,0.02554751,-0.012061629,-0.008222827,0.012662485,0.012484454,0.009702712,-0.024011988,-0.024768623,0.0015077037,0.0061587747,0.02781741,-0.019984027,-0.010770901,0.00317953,-0.016022827,0.009246508,0.039011136,-0.038944375,-0.036919266,-0.018826824,0.0012184025,-0.031978894,0.012818263,-0.0038332392,-0.011961486,0.0030237525,0.006002997,0.029931534,0.018826824,0.046377186,0.0042254645,0.0135303885,-0.014453926,0.009074039,-0.0033186171,-0.009352214,0.030621406,-0.017613985,-0.0022309562,0.021352645,-0.019561203,0.022242801,-0.0023658706,-0.0040335245,-0.03240172,0.028907852,0.030866198,0.015021401,-0.010214553,-0.021486167,0.008222827,-0.014520688,-0.0009694368,0.04372897,-0.005713696,-0.0049209,0.013441373,0.016935239,-0.012951787,0.010342513,0.013953213,-0.011438519,0.08006963,-0.017391445,0.015700147,-0.0073437965,-0.001139123,0.03785393,0.008584453,-0.045041952,0.032268196,-0.0036162634,0.026148366,0.024457067,0.045531537,-0.0017385881,0.021397153,0.025614271,-0.03631841,-0.011538662,-0.008606707,0.018203713,0.0076609147,0.031222261,-0.008795865,-0.003351998,-0.020295583,-0.0037442234,0.029664487,-0.045976616,-0.0020974327,-0.03160058,0.044263065,-0.015933814,-0.0020904783,-0.012150645,-0.009352214,-0.0073326696,-0.021675326,0.02345564,0.028329251,0.006164338,0.045754075,-0.03522797,-0.008690159,0.0399458,0.025814557,0.00190132,-0.00080948666,-0.0651595,-0.032935813,0.027505856,0.018559776,-0.0031183318,0.030398866,0.012317549,0.025124686,0.03331413,-0.007293725,0.004500857,-0.034627113,0.040346373,0.016512414,0.035094444,0.0053298157,0.023589164,-0.0041475757,0.01436491,-0.008222827,0.00074272486,0.016367765,-0.035784315,-0.021797722,-0.011894724,-0.001177372,-0.005824966,0.023121832,0.003029316,0.013430246,-0.046199154,0.015922686,-0.0037831678,0.025169194,0.03522797,-0.031467054,0.025235955,-0.007421685,-0.006002997,-0.009852927,-0.04611014,0.023922972,-0.0189826,-0.030732675,0.024212275,-0.016134098,0.020417979,0.005836093,0.014965767,-0.014976894,-0.013864198,-0.021397153,-0.02605935,-0.02939744,-0.053409427,0.012562343,0.01469872,0.030554643,0.024612844,-0.028796583,0.015600003,-0.014832243,-0.0026565627,-0.0076497877,0.0052018557,0.066717274,-0.012684739,0.02390072,-0.019639092,0.008289589,-0.00051044946,0.013140945,0.028574044,-0.030732675,0.007883454,0.014676466,-0.014142372,0.03030985,-0.0034883032,0.0054383036,-0.029130392,0.030287595,0.01627875,-0.0036162634,-0.004740087,0.02283253,0.025280463,0.025903573,0.0055384464,-0.041614845,-0.005638589,0.0043005715,0.02169758,-0.011661058,-0.03082169,-0.024991162,-0.015677892,0.004584309,0.009524682,0.0291749,0.014209134,0.024612844,0.016434526,0.02928617,0.021085598,0.0076720417,-0.024078751,-0.027327824,0.0094913,0.016211987,-0.004564837,0.01198374,-0.00938003,0.04159259,0.008779175,0.017402573,0.013441373,-0.0030793874,-0.025703287,-0.04366221,-0.03306934,-0.0075663356,0.010164482,0.00069126266,0.010971187,0.01123267,-0.00010631466,0.0035300294,0.010542799,0.004612127,0.0020751788,-0.020651646,0.016423399,0.033514418,0.010893298,0.030487882,-0.015622257,0.029775755,0.008567763,0.004948717,0.01484337,0.028195728,0.0052713994,-0.017224541,-0.025213702,-0.005869474,-0.06391328,-0.015600003,-0.01821484,0.010320259,-0.0110602025,0.014965767,-0.035517268,0.010047649,-0.012428819,0.032846797,0.026037097,-0.023099577,-0.0019430461,-0.010487163,-0.010019831,0.00899615,-0.0052853078,0.007232527,0.014609704,-0.022020262,-0.018548649,0.027038522,-0.0027483602,-0.01748046,-0.023477895,0.014064482,0.024011988,-0.019138379,-0.013930959,-0.041548084,-0.01627875,0.02206477,0.010476037,-0.044841666,-0.021374898,0.0017038163,-0.014053356,0.012417692,0.020584883,-0.021085598,0.014921259,0.005747077,-0.017881032,-0.03456035,-0.053097874,-0.008300715,-0.011054639,-0.03162283,0.0005358329,0.00092006085,0.016078463,-0.0054216133,-0.018804569,-0.022921545,0.010036522,-0.054700155,0.0034410136,-0.02837376,0.01679059,-0.01733581,-0.0043868055,-0.011405138,-0.004434095,0.0074606296,0.025413986,-0.012250788,-0.013675039,0.022276182,0.0022267837,0.014264768,0.011438519,0.011082456,-0.022131532,0.019372044,0.030799437,0.014809989,-0.01627875,0.006153211,-0.026215127,-0.018459633,-0.013919832,0.022409705,0.017881032,0.00246045,0.030131819,-0.004770686,-0.021029962,-0.0050989315,-0.013775182,0.00013326279,0.044441096,-0.007455066,-0.013719547,0.016223114,-0.0066761784,0.0021711488,0.021942373,-0.013474753,-0.028596297,-0.013730674,0.005641371,-0.0006811788,-0.0019889448,0.033803716,0.0028067767,0.005897291,-0.0076275337,0.025480747,0.03943396,0.031266768,-0.04012383,-0.031244515,-0.014720974,0.0027316697,0.021908993,-0.029820263,-0.0040474334,0.022587737,-0.0005236628,-0.0032880178,-0.009574752,-0.041548084,-0.027172046,-0.0028387667,0.0009951679,0.024234528,-0.008473183,0.032780036,0.017213413,0.022164913,-0.026838237,0.035762064,-0.025347224,-0.047000296,0.0135303885,0.03727533,0.0038582748,-0.07579688,0.004381242,0.025970334,0.0020334527,0.016801717,0.009391158,0.021196866,-0.014465054,0.0149991475,0.016134098,-0.009435666,-0.01447618,-0.011961486,-0.015633384,0.0076831686,-0.0026009278,0.0018136952,0.026994014,-0.007143511,-0.024011988,-0.0037803862,0.015811415,-0.044552363,0.029130392,0.018637665,-0.007176892,-0.0029263915,0.022899292,0.010381457,-0.005980743,-0.034360066,0.022120405,-0.008528818,0.01495464,0.033692446,0.0067318133,-0.033047084,-0.002834594,-0.0016899076,0.0041336673,0.019739235,0.006831956,0.00014725841,0.015332957,-0.03378146,-0.03326962,-0.012106137,0.045398016,0.010342513,-0.0045592734,-0.061999444,-0.011388448,-0.019616839,0.013118691,-0.039901294,0.0031350222,0.02872982,0.038543805,0.03211242,0.0029403004,0.009424538,0.0043228255,-0.0052992166,-0.020440234,0.014988021,0.012929533,-0.024256783,0.0019152287,0.012072756,-0.010359203,-0.014131244,0.0069543524,-0.015032529,0.01436491,-0.0069877333,-0.02272126,-0.024390304,0.0040585604,-0.007377177,0.013574896,0.014932386,0.02844052,0.010693013,-0.035895586,-0.008256207,0.0416371,0.0020056353,0.013430246,-0.026704714,0.02939744,-0.06631671,-0.036095873,0.002984808,-0.03716406,-0.02019544,-0.01594494,0.036229394,0.032891307,-0.0026885527,0.019627964,-0.026415413,0.0009965587,0.030131819,-0.014554069,-0.0012851644,-0.0044424403,-0.0020028534,-0.012962913,-0.015711274,0.010492727,-0.032090165,0.008373041,0.0027024613,0.052430253,-0.00822839,0.013441373,-0.006659488,0.027728396,0.0075107007,-0.013630531,0.013396865,-0.0050099157,0.0094913,0.0040724687,-0.002278246,0.0043562064,-0.028907852,-0.01858203,0.028351504,-0.01198374,-0.021185739,-0.0026106639,0.013063055,-0.04944823,0.008912698,0.01108802,0.051362067,0.033336386,0.0064925835,-0.01931641,0.0051712566,0.029775755,-0.018871332,0.013074183,-0.0043422976,0.026838237,-0.01597832,-0.0023018906,0.017324682,-0.008284025,-0.0021516767,0.003727533,-0.01637889,0.03863282,0.001164854,0.031756356,-0.0073215426,0.017714126,0.021753214,0.008367477,0.038766343,-0.0006571863,-0.028306996,0.031511564,-0.012651358,-0.0052380185,0.025970334,0.039567485,0.00258841,0.043973763,0.0063089887,0.011460773,0.030465627,0.00896277,-0.044485603,0.015778035,-0.006314552,-0.0013060274,-0.010642941,0.038165487,0.00021123845,0.032713275,-0.009519118,-0.037141807,-0.039055645,-0.008729103,0.009274324,0.019839376,0.0057192594,-0.009213126,0.052830826,0.008478747,0.019182887,-0.035205714,-0.003491085,0.032935813,0.00061824196,-0.017569477,0.0135081345,0.0053325975,0.001492404,0.0154887345,0.031289022,0.01594494,0.030220835,0.0050349515,-0.011182599,-0.031266768,0.01993952,-0.012417692,0.0543886,-0.039011136,0.0015132672,0.0029820264,0.01719116,0.018292729,-0.0013463626,-0.026370905,0.0325575,-0.022632245,0.002912483,-0.0232331,0.008406421,-0.046822265,-0.0018289947,-0.040413134,-0.018938093,0.01663481,0.02254323,0.007338233,0.011049075,-0.028841091,0.021274755,0.023922972,0.009797292,-0.030688167,-0.017358065,0.0022434741,-0.012684739],
        "Category": "Suite",
        "Tags": [
            "air conditioning",
            "laundry service",
            "24-hour front desk service"
        ],
        "ParkingIncluded": "true",
        "LastRenovationDate": "2018-01-27T00:00:00Z",
        "Rating": 2.7,
        "Address": {
            "StreetAddress": "1100 S Hayes St",
            "City": "Arlington",
            "StateProvince": "VA",
            "PostalCode": "22202",
            "Country": "USA"
        },
        "Location": {
            "type": "Point",
            "coordinates": [
            -77.060066,
            38.863659
            ],
            "crs": {
            "type": "name",
            "properties": {
                "name": "EPSG:4326"
                }
            }
        }
    },
    {
        "@search.action": "mergeOrUpload",
        "HotelId": "13",
        "HotelName": "Luxury Lion Resort",
        "Description": "Unmatched Luxury. Visit our downtown hotel to indulge in luxury accommodations. Moments from the stadium and transportation hubs, we feature the best in convenience and comfort.",
        "DescriptionVector": [-0.0043867915,-0.055055395,0.038483072,0.0319377,-0.03757786,-0.023976486,-0.0436126,0.01048536,0.01361298,-0.0047900747,-0.009980531,0.0060753585,-0.01002115,-0.037740335,0.03226265,0.027086698,-0.01874251,0.007897385,0.024858486,0.017071351,0.034235545,-0.008680741,0.0021788892,-0.0310557,-0.008315175,0.030359384,-0.028084751,0.00700378,0.020181563,0.018870167,0.0082977675,-0.017257035,0.026483223,-0.023593511,0.013438901,0.03834381,0.055055395,0.03713686,0.005698187,-0.021910748,-0.023233749,0.0804013,-0.006237832,0.0064583323,-0.004160489,0.0065975953,-0.018359536,0.032123383,0.005999924,0.04312518,-0.016491087,-0.02583333,-0.018475588,-0.05788708,-0.0023152512,-0.0073751486,0.020494904,-0.033075016,0.059511818,-0.03453728,-0.054034133,-0.010160413,-0.013705823,0.08959267,-0.007050201,0.012475664,-0.0026111854,0.031775225,-0.02420859,-0.025601223,0.041152284,0.009411873,-0.0337017,0.0026155375,0.03411949,0.0010060318,-0.01768643,0.022688301,-0.0069283457,-0.012150717,-0.008622715,-0.018614851,-0.043728653,0.04695492,-0.038483072,-0.019357588,-0.025972592,-0.0003410861,0.0018539417,0.07227761,0.01878893,0.030823594,0.0070676086,-0.025392327,-0.016514298,0.047767285,0.02629754,-0.04389113,-0.000029058505,0.06847109,0.055426765,-0.08207246,0.010920558,0.006208819,-0.009655584,-0.075434245,0.010642031,-0.010113992,0.022699906,0.020030694,-0.06693919,0.010392519,-0.01403077,0.0088025965,0.013972743,0.013636191,-0.00680649,-0.04533018,-0.022073222,-0.03532644,0.020111931,0.010642031,-0.010729071,-0.042359233,-0.02803833,-0.058165606,-0.009707808,0.000060247665,-0.05059897,-0.018556826,-0.057283606,-0.014170033,-0.019984273,-0.0054660817,-0.025299486,0.027458066,-0.027759803,-0.017396297,0.008622715,-0.0019743463,-0.0306147,-0.059372555,0.030800384,-0.024719223,-0.04238244,-0.0020584846,0.02413896,0.032239437,0.002358771,0.03456049,-0.027504487,0.010084978,-0.0017901127,-0.030916436,-0.007160451,-0.029686278,-0.023001643,-0.0038790612,-0.021516168,0.024324644,0.0012243559,0.00021161482,-0.00874457,-0.019995878,0.058212027,-0.004482535,-0.032193016,0.011100439,-0.071488455,-0.0200423,0.036672648,0.0018771522,-0.005497996,0.047117393,0.03794923,0.021249248,-0.040386334,0.019972667,0.025090592,0.0054138578,0.017082956,-0.05524108,0.079472885,-0.013833481,0.006139187,0.04748876,-0.035999544,0.02717954,0.013230006,0.056169502,-0.010891545,0.005825845,-0.038924072,0.027690172,-0.01922993,0.012487269,-0.027690172,0.036951177,0.004189502,0.00006527964,-0.01173873,0.018231878,0.0381117,-0.02717954,-0.014773508,-0.019287957,-0.029570226,0.019937852,-0.024835275,0.007305517,0.011918611,0.011611071,0.024928117,0.02585654,0.023976486,-0.043334074,-0.036626227,-0.026065433,0.042869862,0.03497828,-0.016839245,0.0121159,-0.0091971755,0.056215923,-0.003539607,0.05222371,-0.040316705,-0.01043894,-0.0018539417,-0.010607216,-0.022978432,-0.037809964,-0.033399966,-0.03407307,-0.023535484,0.043821495,0.015168087,-0.027806224,0.03753144,0.022282116,-0.013206796,-0.003475778,0.003969002,0.03107891,0.02413896,-0.03456049,-0.00079568627,0.017280245,0.0535235,0.047952972,-0.02244459,0.034258753,-0.012812217,-0.017373087,0.031287804,0.0101546105,-0.018730905,0.027295593,0.03748502,-0.029593436,0.0332607,0.03632449,-0.017779272,-0.01444856,0.00089795765,-0.0058374503,-0.002830235,-0.0323787,0.0035134952,0.0003051823,-0.07594488,-0.032680437,-0.053430658,0.03242512,-0.040711284,-0.060579505,-0.039945334,0.03154312,0.043960758,0.012208743,-0.040618442,-0.047303077,0.043914337,-0.022827564,-0.027783014,-0.020564536,0.046699602,0.021899143,-0.058026344,-0.025694065,-0.015086849,-0.033933807,-0.018405957,-0.024858486,0.0013774004,-0.012858638,-0.0019685437,-0.045074865,0.030127278,-0.009968926,0.04282344,0.06749625,-0.02541554,-0.02543875,0.013740638,0.02801512,0.033840965,0.012127506,0.045933653,0.0049438444,-0.0034438635,0.02592617,-0.0038906664,-0.0043461733,-0.0077175037,0.0058490555,-0.010688453,-0.003522199,-0.025299486,-0.017129377,0.055798132,0.02134209,-0.028316855,-0.009290018,0.029663067,-0.018904982,-0.0422896,-0.020808248,-0.022932012,-0.030173698,0.018951405,-0.009696202,0.02266509,0.020413669,-0.03284291,-0.015655508,-0.027388435,-0.011048216,0.013984349,0.026042223,-0.024556749,-0.028734645,-0.008448636,-0.02163222,0.0030811988,-0.04790655,0.009231991,0.032564383,0.018011378,0.037763543,-0.03279649,-0.014309296,0.0352336,0.04748876,-0.009997939,-0.01381027,0.056215923,0.033492807,-0.019125484,-0.017767666,-0.01361298,0.0032581792,-0.023976486,-0.0117851505,0.055937394,0.012220348,-0.023222143,-0.0006615003,-0.009887689,-0.009707808,0.008088873,0.0038268375,0.04827792,0.060347397,-0.023314985,0.04345013,0.026947435,0.040502388,-0.028827488,-0.044192865,0.02462638,0.0037368967,0.018429168,-0.026993856,0.0662893,0.01925314,-0.012220348,-0.026576066,-0.0091971755,0.011239703,-0.015121666,-0.023790801,0.019717352,-0.044703495,-0.042405654,0.020402063,0.052966446,0.013311244,0.009539531,-0.027574118,0.023071274,-0.04403039,-0.0009124643,-0.036069177,-0.009226189,-0.029918384,0.011466006,-0.052084446,-0.020030694,0.034189124,-0.007235885,-0.0046856273,-0.0049525485,-0.025578013,-0.00026891584,0.026831381,0.008761978,-0.035512123,0.034189124,0.035651386,0.024278222,0.050970342,0.014634244,0.046978127,-0.025531592,0.02369796,0.058490556,-0.045910444,-0.03460691,0.008773583,-0.050227605,-0.0207038,0.036022756,0.032216225,-0.056308765,0.07009582,-0.033748124,0.0053094104,-0.010206834,-0.012533691,-0.04574797,-0.040618442,-0.032982174,0.010473755,-0.024069328,0.03198412,0.0094234785,0.02889712,0.02286238,-0.00025712923,-0.0068238983,-0.019392405,-0.0007920596,-0.026529646,-0.016258981,0.049949076,0.05658729,-0.021376906,0.029639857,-0.017814089,0.014413744,-0.027899066,0.050088342,0.0072997143,0.00453766,-0.011146861,0.0002939397,0.013183585,0.030684331,-0.030730752,0.0074505825,-0.0134737175,0.05867624,0.05227013,0.0076768855,-0.033957016,-0.0104911635,0.016572325,0.0017233824,-0.024742434,-0.032169804,-0.009667189,-0.035117544,-0.017303456,-0.04827792,-0.015365376,-0.011396374,-0.015168087,-0.08272236,-0.025531592,-0.01024165,-0.0025865242,-0.047767285,-0.010015347,-0.01154144,0.001669708,-0.013496928,0.008988281,0.0070676086,-0.0038732586,-0.022804353,-0.059465397,0.032935753,0.017013324,0.027991908,0.03488544,-0.01768643,-0.038042072,0.045074865,0.010752281,-0.0082977675,0.018638061,-0.019171905,0.03504791,0.007264898,0.03834381,-0.00006364765,0.003577324,0.0050802063,-0.010839321,0.0134156905,-0.004444818,0.0030115673,0.006910938,0.017988168,0.030034436,-0.0049815616,-0.034769386,0.0053297197,0.017303456,-0.0026372974,-0.014320902,-0.027968697,0.017396297,0.0102648605,0.02629754,0.015121666,0.0038268375,0.05380203,-0.019148694,0.042498495,-0.0018220273,-0.0075318194,0.006110174,0.0104911635,-0.04043276,-0.030707542,0.00019529491,0.01883535,0.033051807,-0.009812254,0.018754115,0.011466006,-0.0055734306,-0.01358977,0.00711403,0.010711663,0.01037511,0.02046009,0.012545296,-0.0032088568,-0.0033626268,-0.013543349,0.0075782407,0.026854591,0.043032337,0.02806154,0.045423023,0.00076014514,-0.015817981,-0.002656156,0.042475283,-0.0026909718,0.0003448941,-0.017419508,0.005431266,0.026901014,-0.01856843,-0.019891432,0.0242318,0.021284062,-0.022386564,0.03713686,0.026436802,-0.034258753,-0.0077407146,0.01883535,-0.0025401032,0.011756137,0.04786013,0.00962657,0.014622639,0.011686506,-0.008036649,0.0061507924,-0.00004490242,-0.05268792,-0.013183585,-0.01444856,0.0111642685,0.024835275,-0.02590296,0.014529796,-0.020866273,-0.016317008,0.008355794,0.0022166064,0.005724299,0.011378966,-0.0034670741,0.026227908,-0.0118954005,0.019090667,0.0022427181,-0.030405805,0.039388284,-0.037345756,-0.011831571,0.019612905,-0.011645887,0.034676544,0.029593436,0.08601825,0.019937852,0.004700134,0.010798703,0.013125559,-0.04841718,0.0015391487,-0.01856843,0.00036121398,0.04349655,0.039945334,-0.007792938,-0.005706891,0.043798286,-0.000085407526,0.0077407146,-0.030870015,-0.01599206,-0.031566333,0.0113905715,0.040896967,0.014390534,0.02592617,0.026111854,0.054173395,-0.0034409622,-0.0066440166,0.00024987594,-0.022398168,-0.003794923,0.016758008,0.023883643,0.00131067,0.015005613,0.000107167405,0.0027011263,0.03363207,0.063271925,0.018243482,0.049113497,-0.025044171,-0.0056952857,-0.013671007,0.045005232,0.016224166,0.024278222,-0.019218326,0.017535562,0.07264898,0.015342166,0.014970797,-0.000658599,0.04658355,0.0008246994,-0.00026238788,-0.004270739,0.0011946174,-0.047952972,0.00700378,-0.00605795,-0.06090445,0.01900943,0.01903264,-0.012638138,0.03270365,0.044239286,0.017941745,-0.021620616,-0.00068144687,-0.010113992,0.02281596,0.0034902846,0.0015420502,-0.016096508,0.0078451615,0.036045965,0.005434167,-0.031357437,0.014576218,0.0076884907,0.0059418976,-0.036463756,-0.0036469558,-0.015817981,0.025369117,-0.04054881,0.008408017,0.012545296,0.017013324,-0.016932087,-0.0071372404,-0.01856843,0.022630274,0.011721321,-0.011349953,0.024719223,-0.0041372785,-0.04024707,-0.003916778,-0.009609163,0.00350189,0.027017066,-0.0006611377,-0.0033539226,0.016583929,0.004308456,-0.007949609,0.047419127,0.022769537,0.005268792,-0.034281965,0.021620616,0.022374958,-0.013044322,0.018382747,0.024835275,-0.044657074,0.0013839283,0.0040821536,0.011146861,-0.013554954,-0.029430961,0.036719073,-0.024788855,0.0088084,-0.012754191,-0.060486663,0.014529796,-0.050274026,0.0076652803,0.00010517275,-0.05166666,-0.0011554495,-0.07975141,0.03277328,0.027666962,0.020866273,0.051109605,-0.031728804,0.009127544,-0.002255774,0.03407307,0.015075244,-0.029036382,0.02354709,0.017106166,-0.0440536,-0.020808248,0.00808307,-0.010676848,0.015400192,-0.020320825,0.03247154,-0.019485246,0.0026155375,-0.040664863,-0.016978508,-0.030916436,0.024579959,0.009597558,0.024835275,-0.02847933,-0.020309221,-0.021725064,0.01920672,-0.008007635,0.032634016,0.013554954,0.0050918115,-0.0007768277,0.023373011,-0.007125635,-0.003063791,0.030196909,0.009522123,-0.013659402,-0.0019815997,0.027899066,0.041918233,0.00464791,0.027434856,0.00030608897,-0.012672953,-0.0050657,-0.020169957,-0.019833405,-0.022920405,-0.024974538,0.03409628,0.055287503,0.001724833,-0.021156406,0.00055523956,0.0013157474,-0.02006551,0.011141058,-0.058026344,-0.025020959,-0.0088084,-0.03189128,-0.028966751,0.020135142,0.0034902846,0.0016987212,-0.032123383,0.00832678,-0.007856767,-0.074459404,-0.027458066,0.016792824,0.010026952,0.02850254,0.010009544,-0.0066324114,0.016641956,0.026042223,0.003667265,0.0120114535,0.014053981,-0.008077268,-0.061600767,0.017744455,-0.01903264,-0.011988243,0.01793014,-0.007908991,-0.02046009,-0.0076130563,0.0242318,0.036719073,0.016978508,-0.062343504,0.023373011,-0.023976486,0.0051150224,-0.0032784885,-0.020541325,-0.02182951,-0.025786908,0.021237642,0.010560795,-0.04006139,-0.031171752,0.029918384,-0.00006260499,0.0176168,0.018173851,-0.008750373,-0.033840965,0.0054196604,-0.004444818,-0.010723269,-0.014390534,-0.01574835,0.031752016,0.032076962,0.008512464,-0.020843063,0.027040277,0.0012388624,-0.013032717,0.018533614,-0.020796642,-0.0061682006,-0.018904982,0.043403707,0.00044136288,-0.041221917,0.03230907,0.016966904,0.008953465,-0.035465702,-0.025949381,0.008280359,-0.009725215,0.02977912,-0.027458066,0.038924072,0.0012889102,-0.012255164,-0.029570226,0.004984463,-0.012800612,0.020007484,-0.0014267227,-0.00096396264,0.0142048495,-0.0068355035,0.011210689,0.0077058985,-0.0049786605,-0.01810422,-0.0204833,0.05356992,-0.040363126,-0.014959192,-0.0047494564,0.019125484,-0.016375035,-0.004853904,-0.0400846,0.02416217,0.016375035,0.007653675,-0.043775074,0.014901165,0.014808323,-0.011593664,-0.030452225,0.0052455817,-0.03319107,-0.0042852457,0.035628177,0.040409546,0.042591337,0.011257111,-0.011448598,0.0063248714,-0.017303456,0.036417335,0.013763849,0.013264823,0.0141468225,-0.014181638,-0.0018423364,0.017349876,-0.010845124,0.0072474903,-0.0002625692,0.004920634,-0.018139035,-0.011251308,-0.03451407,-0.017872114,0.004351976,-0.01764001,-0.016467877,-0.0118954005,0.030475436,0.042637758,0.018556826,-0.04187181,0.012197138,0.018173851,-0.0120114535,0.046026498,-0.040316705,-0.019682536,-0.046746023,0.04087376,0.030336173,-0.018638061,-0.02303646,0.045098074,0.021214431,-0.026622487,0.03279649,0.007508609,0.021109983,0.047117393,0.028641803,-0.018580036,0.023906853,-0.00872136,0.029245278,0.04618897,0.06638214,-0.013090744,0.029755909,-0.015922429,0.0014201947,-0.0047784694,-0.011750335,-0.010560795,0.0057562133,0.039922126,0.012858638,0.0014883757,-0.015260928,-0.0038674558,-0.020274404,-0.01601527,-0.011802559,-0.0072939117,0.01317198,0.054684028,0.016908877,-0.02182951,0.0015899219,0.029059593,-0.014367322,0.033980228,-0.005178851,0.0065975953,0.01491277,-0.012498874,0.00028650506,0.0588155,0.019102274,0.018359536,0.011976638,0.020274404,0.002859248,-0.0213653,0.024510328,0.030962858,0.017872114,-0.0017698035,0.021260852,0.0003619393,0.021109983,0.028781068,-0.003327811,0.00453766,-0.0030724949,-0.014460165,-0.015284139,0.026483223,0.012371217,0.011907006,0.0108857425,0.025113802,-0.010073373,0.004337469,0.00014488453,0.018498799,0.04015423,0.0007010308,0.027713383,-0.0063074636,0.004380989,0.009638175,-0.011129453,0.006446727,-0.011007598,-0.020982327,0.039341863,-0.0207038,-0.004674022,0.0006799962,-0.01570193,-0.035604965,-0.013508533,0.00007747424,-0.04010781,-0.031566333,-0.023233749,0.0050918115,0.019810194,0.04837076,0.020529721,-0.012231953,-0.012800612,0.008065662,0.0005131705,-0.009284215,0.016688377,0.024696013,-0.0061856085,0.015817981,-0.022108037,0.014158428,-0.037392177,0.013322849,0.019125484,0.019856615,0.0057301014,-0.010102387,0.033817753,0.01788372,-0.02264188,-0.0071662534,0.009725215,-0.018614851,0.016084902,0.031357437,0.0137522435,0.014773508,0.015318955,0.03363207,0.02286238,-0.0032175607,0.0141468225,0.0047320486,-0.009336439,-0.007763925,-0.008814202,-0.0068471087,0.032517962,0.01143119,0.006382898,0.04043276,0.026692118,0.018429168,0.0124060325,0.022189274,-0.04187181,-0.017407903,0.030312963,-0.0381117,0.012893454,-0.0068935296,-0.004946746,0.004517351,-0.007850965,0.0209243,-0.012464059,0.033608858,0.011640085,-0.02119122,-0.031798437,-0.009643978,-0.025810119,-0.04574797,0.01744272,0.042962704,-0.008547281,-0.0058490555,0.0079031885,-0.021284062,0.011083032,-0.017999772,0.03890086,-0.02462638,0.020030694,0.05779424,0.017106166,0.003577324,0.016932087,0.0008566139,-0.013195191,-0.00016664442,-0.010763887,0.019171905,0.018440772,-0.0009987785,-0.006910938,-0.028200803,-0.0061043715,-0.0043693837,-0.04136118,0.0033713307,-0.009278413,0.00852407,-0.011251308,0.036974385,0.030939646,0.0021817905,0.010990189,-0.01748914,0.0007586944,-0.007508609,0.034792595,0.02418538,0.021667037,0.017419508,-0.0022398168,-0.011117848,-0.040943388,0.003339416,0.0009102883,-0.014808323,0.006429319,0.010201031,-0.0018452378,0.015876008,0.0034960872,-0.0064815427,-0.019160299,0.028200803,0.025044171,-0.03790281,-0.0310557,0.013624585,0.0070269904,0.026483223,0.001724833,-0.017999772,0.0006977668,-0.018278299,-0.01643306,0.03291254,-0.0153305605,-0.0014419546,-0.045074865,-0.011222295,-0.025299486,0.0007184387,-0.016026877,0.008274557,-0.001423096,-0.05686582,0.05524108,-0.053337816,0.02156259,0.014761902,-0.013891507,-0.019496853,-0.03103249,-0.025020959,0.015539455,0.01616614,0.044216074,-0.013833481,0.02332659,0.005315213,0.017964955,0.0113905715,0.00581424,-0.04607292,-0.012556901,0.012452453,0.023222143,0.00021832412,-0.005515404,-0.009539531,-0.018231878,-0.0043432717,-0.012104295,-0.011907006,0.022386564,0.008141096,-0.015678719,0.0014064135,-0.031868067,0.0022934913,-0.019392405,-0.032564383,0.042869862,-0.022792747,0.00095235737,-0.0022209582,-0.022525826,-0.030243332,-0.011930216,-0.020030694,-0.021574195,0.028386489,-0.020668983,0.00061616726,-0.0046014893,-0.017698035,0.020854669,-0.032216225,0.0148779545,0.025624434,0.026529646,0.0051730485,-0.0065685823,0.015249323,-0.020993931,-0.015736744,0.042080704,-0.019438826,0.026460012,0.0075666355,0.014494981,-0.039550755,0.0021020044,-0.03140386,0.006179806,-0.03063791,-0.0025589617,0.038088493,0.004645009,-0.010694255,0.0043983967,-0.0061043715,0.0032639818,0.06322551,0.0035251004,-0.02347746,-0.014460165,0.03797244,-0.013404085,-0.018046193,0.011442795,-0.001568162,-0.024579959,0.023663143,-0.023535484,0.01984501,-0.03230907,0.0071372404,-0.003815232,-0.0004167017,-0.020135142,-0.009290018,0.012197138,0.021260852,0.0043113576,-0.02720275,-0.017547166,-0.02673854,0.024812065,-0.01768643,-0.008820005,-0.01361298,0.042800233,-0.012197138,0.02636717,-0.012823822,0.012765796,0.023198932,0.013682612,-0.012301585,-0.008500859,0.0003528727,0.017732851,0.008535676,0.017129377,-0.013624585,-0.017802482,-0.0086923465,0.0024748235,-0.0097020045,0.0057272003,0.008129491,0.010235847,0.008872228,-0.0082977675,-0.004584081,0.02636717,0.00034144876,0.034235545,-0.032494754,-0.016270587,0.025624434,-0.032076962,0.013601375,0.016096508,0.021678641,0.02847933,0.025299486,-0.0069283457,-0.003731094,-0.0032494753,0.024371065,-0.006191411,0.036092386,-0.003971903,-0.021887537,0.04971697,-0.0127425855,0.0004464402,-0.0021194122,-0.0076362668,-0.004499943,0.026251119,-0.029941594,0.01154144,0.041593283,0.019404009,-0.004993167,0.021295669,-0.023930065,0.0064061084,-0.012498874,-0.042452075,-0.0048161866,0.0012330598,0.009568544,0.006504753,0.0053877463,-0.009638175,-0.015063639,-0.014866349,0.029222067,-0.017848903,-0.026622487,-0.0005124452,-0.00420691,-0.011233901,-0.04407681,0.0032436727,0.0015768659,0.010334492,-0.0008406567,0.0012410384,-0.010897348,0.0002165108,0.03630128,-0.002846192,-0.028757857,-0.0015826685,0.010891545,0.007671083,0.005585036,0.0107754925,-0.0008587899,0.0068819243,0.022154458,-0.00047726667,-0.011907006,0.020692194,-0.017732851,0.011918611,-0.005149838,0.0027185343,-0.023361407,-0.022015195,-0.004444818,-0.028804278,0.0047320486,-0.009498913,-0.043334074,-0.038993705,-0.019531667,-0.017535562,0.009620768,-0.024115749,0.03158954,0.034374807,0.010734874,-0.023233749,-0.0008246994,0.0041662916,-0.009115939,0.026576066,-0.01574835,0.0051817526,-0.02205001,0.0033162057,0.024719223,0.03504791,0.015690323,-0.023454249,-0.0109379655,0.0137522435,0.005094713,0.013380875,0.008280359,0.058954764,0.006284253,-0.007409964,-0.012638138,-0.008077268,0.0211448,0.029454172,0.021759879,0.002065738,0.016792824,0.011216492,-0.010003742,0.03532644,-0.025137013,0.0149359815,-0.009614965,0.025299486,-0.07817309,0.0062958584,-0.005149838,0.0048451996,0.008883833,0.02980233,0.029268488,0.04447139,-0.008228135,-0.006545372,-0.011222295,0.045098074,-0.01945043,0.0015754153,0.02636717,-0.012301585,0.019891432,-0.0049351407,0.022084827,0.004906127,0.02590296,0.008895438,-0.029245278,-0.023396222,0.044192865,0.029245278,0.02286238,-0.021841116,0.012336401,-0.042312812,0.013125559,-0.010769689,-0.012231953,-0.03534965,-0.03110212,-0.005817141,0.04618897,-0.02160901,0.004856805,0.014065586,0.006777477,0.015736744,-0.008408017,0.018487193,0.042846654,0.004172094,0.022212485,-0.0030115673,0.010949572,-0.0076420694,0.0072765034,-0.043241233,0.02889712,0.026692118,-0.009539531,0.0152725335,0.012266769,0.007235885,0.030080857,-0.006261043,0.01270777,0.019311167,-0.013346059,0.013856691,-0.0052658906],
        "Category": "Luxury",
        "Tags": [
            "bar",
            "concierge",
            "restaurant"
        ],
        "ParkingIncluded": "false",
        "LastRenovationDate": "2020-03-18T00:00:00Z",
        "Rating": 4.1,
        "Address": {
            "StreetAddress": "3 Cityplace Dr",
            "City": "St. Louis",
            "StateProvince": "MO",
            "PostalCode": "63141",
            "Country": "USA"
        },
        "Location": {
            "type": "Point",
            "coordinates": [
            -90.44081,
            38.67219
            ],
            "crs": {
            "type": "name",
            "properties": {
                "name": "EPSG:4326"
                }
            }
        }
    }
]


In [93]:
import pandas as pd
dataset = pd.read_csv('hotels.csv')
dataset

,Unnamed: 0,@search.action,HotelId,HotelName,Description,Category,Tags,ParkingIncluded,LastRenovationDate,Rating,Address,Location
0,0,mergeOrUpload,1,Stay-Kay City Hotel,This classic hotel is fully-refurbished and id...,Boutique,"['view', 'air conditioning', 'concierge']",False,2022-01-18T00:00:00Z,3.6,"{'StreetAddress': '677 5th Ave', 'City': 'New ...","{'type': 'Point', 'coordinates': [-73.975403, ..."
1,1,mergeOrUpload,2,Old Century Hotel,The hotel is situated in a nineteenth century ...,Boutique,"['pool', 'free wifi', 'air conditioning', 'con...",False,2019-02-18T00:00:00Z,3.6,{'StreetAddress': '140 University Town Center ...,"{'type': 'Point', 'coordinates': [-82.452843, ..."
2,2,mergeOrUpload,3,Gastronomic Landscape Hotel,The Gastronomic Hotel stands out for its culin...,Suite,"['restaurant', 'bar', 'continental breakfast']",True,2015-09-20T00:00:00Z,4.8,"{'StreetAddress': '3393 Peachtree Rd', 'City':...","{'type': 'Point', 'coordinates': [33.847848, -..."
3,3,mergeOrUpload,4,Sublime Palace Hotel,Sublime Palace Hotel is located in the heart o...,Boutique,"['concierge', 'view', 'air conditioning']",True,2020-02-06T00:00:00Z,4.6,"{'StreetAddress': '7400 San Pedro Ave', 'City'...","{'type': 'Point', 'coordinates': [-98.498923, ..."
4,4,mergeOrUpload,48,Nordick's Valley Motel,Only 90 miles (about 2 hours) from the nation'...,Boutique,"['continental breakfast', 'air conditioning', ...",False,2018-02-19T00:00:00Z,4.5,"{'StreetAddress': '1401 I St NW', 'City': 'Was...","{'type': 'Point', 'coordinates': [-77.03241, 3..."
5,5,mergeOrUpload,49,Swirling Currents Hotel,"Spacious rooms, glamorous suites and residence...",Suite,"['air conditioning', 'laundry service', '24-ho...",True,2018-01-27T00:00:00Z,2.7,"{'StreetAddress': '1100 S Hayes St', 'City': '...","{'type': 'Point', 'coordinates': [-77.060066, ..."
6,6,mergeOrUpload,13,Luxury Lion Resort,Unmatched Luxury. Visit our downtown hotel to ...,Luxury,"['bar', 'concierge', 'restaurant']",False,2020-03-18T00:00:00Z,4.1,"{'StreetAddress': '3 Cityplace Dr', 'City': 'S...","{'type': 'Point', 'coordinates': [-90.44081, 3..."


## Upload Encrypted Data to Azure AI Search

In this step, we ingest documents into the search index using the `SearchIndexingBufferedSender`.

> ⚠️ **Training context:**  
For demonstration and training purposes, we upload the encrypted documents directly from memory using the SDK. This is practical for small-scale examples and local workflows.

> **Production best practice:**  

In real-world, production-grade systems, it is recommended to store the data in a data storage solution (e.g., Blob, ADLS) and configure the AI Search service to connect to this data source using an **indexer**. The indexer automatically pulls, transforms, and loads the data into the index, supporting scalable and secure data ingestion pipelines. For more information refer to ai-search-rest-api.ipynb.


In [42]:
from azure.core.exceptions import HttpResponseError



# Create a SearchIndexingBufferedSender
batch_client = SearchIndexingBufferedSender(
    endpoint=endpoint,
    index_name=os.getenv("AZURE_SEARCH_INDEX_NAME_HOTEL"),
    credential=credential)

try:
    # Add upload actions for all documents in a single call
    batch_client.upload_documents(documents=documents)

    # Manually flush to send any remaining documents in the buffer
    batch_client.flush()
except HttpResponseError as e:
    print(f"An error occurred: {e}")
finally:
    # Clean up resources
    batch_client.close()

print(f"Uploaded {len(documents)} documents in total")

Uploaded 7 documents in total


## Generate embeddings for documents using azure openAI

Before indexing vector data into Azure AI Search, we need to convert textual content into high-dimensional **vector embeddings** using an embedding model.

In this example, we:

1. Extract a sample document's text content (`Description` field).
2. Use Azure OpenAI's embedding model to generate a numerical vector representation.
3. Print the result to understand the format and structure of the embedding.

This vector can then be stored in the index's vector field (e.g., `DescriptionVector`) to enable **semantic** or **vector-based search**.


In [44]:
# Example function to generate document embedding
def generate_embeddings(text, model):
    # Generate embeddings for the provided text using the specified model
    embeddings_response = azure_openai_client.embeddings.create(model=model, input=text)
    # Extract the embedding data from the response
    embedding = embeddings_response.data[0].embedding
    return embedding


first_document_content = documents[0]["Description"]
print(f"Content: {first_document_content[:100]}")

content_vector = generate_embeddings(
    first_document_content, model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME")
)
print(content_vector)

Content: This classic hotel is fully-refurbished and ideally located on the main commercial artery of the cit
[-0.014694961719214916, 0.0166377741843462, -0.008899122476577759, -0.02247924916446209, -0.013078124262392521, 0.010372530668973923, -0.009753177873790264, 0.011337417177855968, -0.009055591188371181, -0.012960773892700672, 0.015946706756949425, 0.014603688381612301, 0.007119297981262207, 0.005091732367873192, -0.029468156397342682, -0.028007788583636284, 0.01538602914661169, -0.006072917953133583, -0.008951278403401375, -0.022244546562433243, -0.0359615832567215, 0.020484281703829765, -0.02627360075712204, -0.01551641896367073, -0.015685925260186195, 0.01976713538169861, 0.004583211150020361, 0.001962370704859495, 0.004863549955189228, -0.011552561074495316, -0.015542496927082539, -0.023248551413416862, -0.025843312963843346, -0.017654815688729286, 0.006190268788486719, 0.001019487390294671, -0.014668883755803108, 0.0002950075431726873, -0.001026006881147623, -0.0003905590274

# 🔍 Comparing Search Techniques in Azure AI Search

In this section we will explore the three types of search methods:

- **Keyword (Lexical) Search**
- **Pure Vector (Semantic) Search**
- **Hybrid Search (Keyword + Vector)**

## 🔍 Keyword Search with Azure AI Search

Lexical search — also known as **keyword search** — matches query terms directly to terms in the index using inverted indexes and token matching.

### When to Use Keyword Search
✅ When the user knows the **exact term** or **phrase** to look for (e.g., *"GDPR"*, *"Samsung S23"*).  
✅ When you want **precise filtering** or are working with structured data or well-known taxonomies.  
✅ For high-precision use cases where **exact term matching** is more valuable than semantic meaning.

### When Not to Use It
🚫 When queries are ambiguous, vague, or expressed in **natural language**.  
🚫 When you're trying to find **semantically similar** results that don't share the same exact terms.

This example shows how to perform a lexical search on the `HotelName`, `Description`, and `Category` fields of our hotel documents using the query `"gastronomic"`. Results are scored and sorted based on exact keyword matches.


In [56]:


from azure.search.documents import SearchClient
import os

search_client = SearchClient(
    endpoint=endpoint,
    index_name=os.getenv("AZURE_SEARCH_INDEX_NAME_HOTEL"),
    credential=credential
)

query = "gastronomic"

results = search_client.search(
    search_text=query,
    select='*'
)

print("🔍 Keyword Search Results:\n")
for result in results:
    print(f"HotelName: {result['HotelName']}")
    print(f"Description: {result['Description']}\n")
    print(f"Category: {result['Category']}\n")
    print(f"Score: {result['@search.score']}")


🔍 Keyword Search Results:

HotelName: Gastronomic Landscape Hotel
Description: The Gastronomic Hotel stands out for its culinary excellence under the management of William Dough, who advises on and oversees all of the Hotel’s restaurant services.

Category: Suite

Score: 0.5301874


## 🧠 Vector Search: semantic similarity based on embeddings

Vector search allows us to search using **semantic meaning** rather than exact keywords. It relies on **embeddings** — high-dimensional representations of text — to capture the contextual meaning of a query and match it to similar content.

### When to use vector search
✅ When queries are written in **natural language** (e.g., “historic hotel walk to restaurants and shopping”).  
✅ When you're looking for results that are **contextually or conceptually** similar, not just exact matches.  
✅ In discovery or exploration scenarios where **intent matters** more than wording.

### When not to use it
🚫 When you require **precise keyword matches** (e.g., compliance codes, product names).  
🚫 When your data is not embedded or doesn’t include vector fields.

This example shows how to generate an embedding from a user query, then perform a **k-NN (nearest neighbor)** search over the vector field `DescriptionVector`. The results are hotels whose descriptions are semantically close to the query.


In [94]:

from azure.search.documents.models import VectorizedQuery


query = "historic hotel walk to restaurants and shopping"

    
vector_query = VectorizedQuery(
    vector=generate_embeddings(query, model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME")),
    k_nearest_neighbors=3,
    fields="DescriptionVector"
)

results = search_client.search(
    vector_queries=[vector_query],
    select=["HotelId", "HotelName", "Description", "Category", "Tags"],
    top=5,
    include_total_count=True
)

print(f"Total results: {results.get_count()}")
for result in results:
    print(f"HotelName: {result['HotelName']}")
    print(f"Description: {result['Description']}\n")
    print(f"Category: {result['Category']}\n")
    print(f"Score: {result['@search.score']}")


Total results: 3
HotelName: Gastronomic Landscape Hotel
Description: The Gastronomic Hotel stands out for its culinary excellence under the management of William Dough, who advises on and oversees all of the Hotel’s restaurant services.

Category: Suite

Score: 0.50744194
HotelName: Stay-Kay City Hotel
Description: This classic hotel is fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.

Category: Boutique

Score: 0.50744194
HotelName: Swirling Currents Hotel
Description: Spacious rooms, glamorous suites and residences, rooftop pool, walking access to shopping, dining, entertainment and the city center. Each room comes equipped with a microwave, a coffee maker and a minifridge. In-room entertainment includes complimentary W-Fi and flat-screen TVs.

C

### 🔍 Vector search with filters

In real-world scenarios, vector search often needs to be **combined with filters** to narrow down results based on structured attributes.

This example performs a **semantic search** and filters for hotels that offer `"free wifi"` using a filter on the `Tags` field.


In [54]:

from azure.search.documents.models import VectorizedQuery


vector_query = VectorizedQuery(
    vector=vector,
    k_nearest_neighbors=5,
    fields="DescriptionVector",
    kind="vector",
    exhaustive=True
)

results = search_client.search(
    vector_queries=[vector_query],
    filter="Tags/any(tag: tag eq 'free wifi')",  # <--- NOTICE THE FILTER IS ADDED HERE
    select=["HotelId", "HotelName", "Description", "Category", "Tags"],
    top=5,
    include_total_count=True
)

print(f"Total results: {results.get_count()}")
for result in results:
    doc = result  # result is a dict-like object
    print(f"- HotelId: {doc['HotelId']}, HotelName: {doc['HotelName']}, Category: {doc.get('Category')}")


Total results: 2
- HotelId: 48, HotelName: Nordick's Valley Motel, Category: Boutique
- HotelId: 2, HotelName: Old Century Hotel, Category: Boutique


### 🔍 Semantic Hybrid Search (Keyword + Vector)

This example demonstrates a **semantic hybrid search**, where we combine:

- **Keyword search** via `search_text`: helps match specific terms or exact phrases.
- **Vector search** via `VectorizedQuery`: captures semantic meaning and retrieves contextually similar documents.

The search results are based on a combination of **lexical relevance and vector similarity**,

Ideal for queries like _"hotel with walking distance to restaurants"_, where users expect both precise terms ("hotel", "restaurants") and contextual understanding ("walking distance").

In [96]:
query = "hotel with walking distance to monuments"

    
vector_query = VectorizedQuery(
    vector=generate_embeddings(query, model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME")),
    k_nearest_neighbors=3,
    fields="DescriptionVector"
)

results = search_client.search(
    include_total_count=True,
    search_text=query,  # keyword part
    select=["HotelId", "HotelName", "Description", "Category", "Tags"],
    top=5,
    vector_queries=[vector_query]
)

print(f"Total hybrid results: {results.get_count()}")
for result in results:
    doc = result
    score = result.get("@search.score", "N/A")
    print(f"- Score: {score}")
    print(f"  HotelId: {doc['HotelId']}")            
    print(f"  HotelName: {doc['HotelName']}")
    print(f"  Description: {doc.get('Description')}")
    print(f"  Category: {doc.get('Category')}")
    print(f"  Tags: {doc.get('Tags', 'N/A')}\n")

Total hybrid results: 7
- Score: 0.03205128386616707
  HotelId: 3
  HotelName: Gastronomic Landscape Hotel
  Description: The Gastronomic Hotel stands out for its culinary excellence under the management of William Dough, who advises on and oversees all of the Hotel’s restaurant services.
  Category: Suite
  Tags: ['restaurant', 'bar', 'continental breakfast']

- Score: 0.03201844170689583
  HotelId: 1
  HotelName: Stay-Kay City Hotel
  Description: This classic hotel is fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.
  Category: Boutique
  Tags: ['view', 'air conditioning', 'concierge']

- Score: 0.0320020467042923
  HotelId: 49
  HotelName: Swirling Currents Hotel
  Description: Spacious rooms, glamorous suites and residences, rooftop pool, walk

###  Summary Table

| Search Type       | Description                                 | Best Use Case Example                           | Avoid When...                                 |
|-------------------|---------------------------------------------|--------------------------------------------------|------------------------------------------------|
| Keyword Search     | Exact term matching (TF-IDF, inverted index) | Legal docs, product names, structured filters    | Semantic queries or vague natural language     |
| Vector Search      | Embedding-based semantic similarity         | Blogs, unstructured content, FAQ search          | Needing strict keyword matching                |
| Hybrid Search      | Combines keyword and semantic matching      | General-purpose web search, documentation sites  | Index doesn’t support vector fields            |

## 🔄 Reranking: Refining search results for better relevance

**What is Reranking?**  
Reranking is a technique that takes an initial set of search results and reorders them to improve their relevance to the user’s query. Unlike simple ranking based only on keyword matches or vector similarity, reranking uses advanced models (often based on large language models or semantic understanding) to better assess how well each result matches the query intent.

**How is Reranking Done?**  
1. First, a hybrid search (combining keyword and vector search) retrieves a candidate list of documents.  
2. Then, a reranker model scores each candidate based on deeper semantic analysis.  
3. The results are reordered using these reranker scores to surface the most contextually relevant documents at the top.

**Why Use Reranking?**  
- Improves precision by considering the nuanced meaning of queries and documents.  
- Especially useful for complex or natural language queries where intent understanding is key.



In [78]:
## Reranking 


query = "hotel with walking distance to monuments"

    
vector_query = VectorizedQuery(
    vector=generate_embeddings(query, model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME")),
    k_nearest_neighbors=5,
    fields="DescriptionVector"
)

results = search_client.search(
            include_total_count=True,
            search_text=query,
            select=[
                "HotelId", "HotelName", "Category", "Description"
            ],
            query_type="semantic",
            semantic_configuration_name="my-semantic-config",
            top=5,            
            vector_queries=[vector_query]
        )

print(f"Total semantic hybrid results: {results.get_count()}")
for result in results:
    doc = result
    score = result.get("@search.score", "N/A")
    reranker_score = result.get("@search.reranker_score", "N/A")
    print(f"- Score: {score}")
    print(f"  Re-ranker Score: {reranker_score}")
    print(f"  HotelId: {doc['HotelId']}")
    print(f"  HotelName: {doc['HotelName']}")
    print(f"  Description: {doc.get('Description')}")
    print(f"  Category: {doc.get('Category')}")


Total semantic hybrid results: 7
- Score: 0.03253968432545662
  Re-ranker Score: 2.943596839904785
  HotelId: 4
  HotelName: Sublime Palace Hotel
  Description: Sublime Palace Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 19th century resort, updated for every modern convenience.
  Category: Boutique
- Score: 0.0320020467042923
  Re-ranker Score: 2.166522979736328
  HotelId: 49
  HotelName: Swirling Currents Hotel
  Description: Spacious rooms, glamorous suites and residences, rooftop pool, walking access to shopping, dining, entertainment and the city center. Each room comes equipped with a microwave, a coffee maker and a minifridge. In-room entertainment includes complimentary W-Fi and flat-screen TVs.
  Category: Suite
- Sco

## From search to smart answers: Introducing Retrieval-Augmented Generation (RAG)

So far, we have explored how Azure AI Search can find and rank relevant documents using keyword, vector, and hybrid semantic search—with reranking to boost result quality.

But what if we want a more interactive, natural language experience? Instead of just listing search results, we can generate precise, friendly, and concise answers grounded in real data.

This is where **Retrieval-Augmented Generation (RAG)** comes in. RAG combines the power of:

- **Vector search** to retrieve relevant documents based on your query, and  
- **Large Language Models (LLMs)** to generate answers by reasoning over these documents.

By grounding the generation on trusted sources from your search index, RAG produces accurate, context-aware responses while minimizing hallucinations.

Below, we demonstrate how to implement a simple RAG pipeline using Azure Cognitive Search and Azure OpenAI, answering user queries with factual support from indexed hotel data.


In [85]:
# Create an AzureOpenAI client instance
azure_openai_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("api_version"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)


# The query is sent to the search engine, but it's also passed in the prompt

query = "Can you recommend a hotel in New york? "

vector_query = VectorizedQuery(
    vector=generate_embeddings(query, model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME")),
    k_nearest_neighbors=5,
    fields="DescriptionVector"
)

results = search_client.search(
    include_total_count=True,
    search_text=query,  # keyword part
    select=["HotelId", "HotelName", "Description", "Category", "Address" ,"Tags"],
    top=5,
    vector_queries=[vector_query])

# The prompt includes the query and the source, which are specified further down in the code.


GROUNDED_PROMPT="""
You are a friendly assistant that answer questions.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""


# Retrieve the selected fields from the search index related to the question
sources_formatted = "\n".join([f'{document["HotelName"]}:{document["Description"]}:{document["Category"]}:{document["Address"]}' for document in results])

# Call the Azure OpenAI API

response = azure_openai_client.chat.completions.create(
    messages=[
        
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=os.getenv("deployment")
)

print(response.choices[0].message.content)

- **Stay-Kay City Hotel**
  - Location: 677 5th Ave, New York, NY 10022, USA
  - Description: A classic, fully-refurbished hotel located on the main commercial artery of New York. It's in the heart of the city, just minutes away from Times Square and the historic city center. This prime location makes it ideal for experiencing the attractions of New York City.
